In [1]:
#Copyright 2023 Alvin Hilario

#Package Versions
#Python 3.8.8
#Jupyter Notebook 6.3.0
#Bokeh 2.3.2
#Pandas 1.2.4
#Numpy 1.20.1
#Sklearn 0.24.1
#itertools

from bokeh.plotting import figure
from bokeh.io import output_notebook, show, reset_output, curdoc, push_notebook
from bokeh.layouts import row, column, gridplot
from bokeh.models import ColumnDataSource, Div, Select, Slider, Band, TextInput, CustomJS, RadioButtonGroup, Span, LabelSet
from bokeh.palettes import inferno, Set2
from bokeh.application import Application
from bokeh.application.handlers import FunctionHandler
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import itertools
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
"""
This code is meant to explore the relationships between different arrow and bow variables 
such as Arrow Length, Arrow GPI, Arrow Spine, Arrow Component weight, etc. and the Bow Poundage
with one critical assumption/restraint:

That there is an optimal Point Weight for a given Arrow Length, Arrow Spine, Bow Poundage, and Bow IBO,
because an arrow is similar to a mass+spring system and there's a sweet spot between under and over spined
arrow setups.

Arrow Manufacturer's recommended Spine for a given Bow IBO, Bow Poundage, Arrow Length, and Point Weight
are confusing, much varied between brands, and do not offer a high resolution nor intuitively visual interface
to explore arrow dynamics.

Although one could make an far underspined or far overspined arrow setup, and still have a successful hunt,
I'm more interested in geeking out and providing a tool for fellow archers and hunters.

These values aren't meant to be exact, but it should get you directionally where you want to go, and in
the ballpark.

Not a coder by profession so please excuse the messy code.
"""

"\nThis code is meant to explore the relationships between different arrow and bow variables \nsuch as Arrow Length, Arrow GPI, Arrow Spine, Arrow Component weight, etc. and the Bow Poundage\nwith one critical assumption/restraint:\n\nThat there is an optimal Point Weight for a given Arrow Length, Arrow Spine, Bow Poundage, and Bow IBO,\nbecause an arrow is similar to a mass+spring system and there's a sweet spot between under and over spined\narrow setups.\n\nArrow Manufacturer's recommended Spine for a given Bow IBO, Bow Poundage, Arrow Length, and Point Weight\nare confusing, much varied between brands, and do not offer a high resolution nor intuitively visual interface\nto explore arrow dynamics.\n\nAlthough one could make an far underspined or far overspined arrow setup, and still have a successful hunt,\nI'm more interested in geeking out and providing a tool for fellow archers and hunters.\n\nThese values aren't meant to be exact, but it should get you directionally where you wa

In [4]:
#Read csv files that I've compiled on the various Arrow Spine Charts and GPIs from popular arrows. You can see them in the github repository
#dataset= pd.read_csv("ArrowSpine2.csv")#with upper/lower bounds
dataset = pd.read_csv("ArrowSpine3.csv")#with upper/lower bounds,without 150 spine
datasetArrowGPIs = pd.read_csv("ArrowGPIs.csv")


In [5]:
dataset.head #Display Arrow Spine File Contents

<bound method NDFrame.head of        Shaft  TotalPointWeight  ArrowLength  Spine  LowerBoundPoundage  \
0     Sirius               150           25    400                  40   
1     Sirius               150           25    350                  65   
2     Sirius               150           26    400                  35   
3     Sirius               150           26    350                  60   
4     Sirius               150           26    300                  70   
..       ...               ...          ...    ...                 ...   
167  Victory               150           31    250                  74   
168  Victory               150           32    200                  90   
169  Victory               150           32    250                  72   
170  Victory               150           32    300                  60   
171  Victory               150           32    250                  55   

     UpperBoundPoundage  NominalPoundage  
0                    64             52

In [6]:
datasetArrowGPIs.head #Display Arrow GPI File Contents

<bound method NDFrame.head of         Shaft  Spine        Brand     OD    GPI             Arrow Name  \
0     Rampage    150  Black Eagle  0.307  16.00    Black Eagle Rampage   
1      Apollo    200       Sirius  0.285  12.60          Sirius Apollo   
2      Vulcan    200       Sirius  0.316  12.15          Sirius Vulcan   
3     Spartan    200  Black Eagle  0.297  11.60    Black Eagle Spartan   
4     Kinetic    200      GoldTip  0.282  11.60        GoldTip Kinetic   
..        ...    ...          ...    ...    ...                    ...   
72    Spartan    400  Black Eagle  0.272   7.20    Black Eagle Spartan   
73    Rampage    400  Black Eagle  0.253   7.00    Black Eagle Rampage   
74  Carnivore    400  Black Eagle  0.287   6.80  Black Eagle Carnivore   
75      Orion    400       Sirius  0.221   6.71           Sirius Orion   
76     RIP XV    400      Victory  0.247   5.90         Victory RIP XV   

    Unnamed: 6  
0          NaN  
1          NaN  
2          NaN  
3          Na

In [7]:

#Function to derive the linear relationship between Arrow Length and Poundage for a given Spine
def performLinRegbyArrowLength(dataset, arrowLength, poundageType):
    reg = LinearRegression()
    x=dataset.loc[dataset['ArrowLength'] == arrowLength,'Spine'].to_frame()
    y=dataset.loc[dataset['ArrowLength'] == arrowLength,str(poundageType)+'Poundage'].to_frame()

    df = pd.concat([x,y],axis=1)

    reg.fit(x,y)

    x_test = pd.DataFrame(np.linspace(150,400,30).reshape(30,1))
    y_pred = pd.DataFrame(reg.predict(x_test))

    df2 = pd.concat([x_test,y_pred],axis=1)
    df2.columns = ["Spine", str(poundageType)+'Poundage']

    return df, df2, reg.coef_, reg.intercept_

#Function to derive the linear relationship between Arrow Length and Poundage for a given Spine and Brand of Arrow
def performLinRegbyArrowLengthAndBrand(dataset, arrowLength, brand, poundageType):
    reg = LinearRegression()
    x=dataset.loc[(dataset['ArrowLength'] == arrowLength) & (dataset['Shaft'] == brand), 'Spine'].to_frame()
    y=dataset.loc[(dataset['ArrowLength'] == arrowLength) & (dataset['Shaft'] == brand), str(poundageType)+'Poundage'].to_frame()
    
    
    df = pd.concat([x,y],axis=1)

    reg.fit(x,y)

    x_test = pd.DataFrame(np.linspace(150,400,30).reshape(30,1))
    y_pred = pd.DataFrame(reg.predict(x_test))

    df2 = pd.concat([x_test,y_pred],axis=1)
    df2.columns = ["Spine", str(poundageType)+'Poundage']

    return df, df2, reg.coef_, reg.intercept_

#Function to derive the linear relationship between Arrow Length and Poundage for a given Spine
def performLinRegbyArrowLengthAll(dataset, arrowLength):
    reg = LinearRegression()
    poundageTypes = ['Nominal', 'LowerBound', 'UpperBound']
    
    d = {'Spine': [],'Poundage': []}
    df4 = pd.DataFrame(data=d)
    df3 = pd.DataFrame(data=d)
    
    for poundageType in poundageTypes:
        x=dataset.loc[dataset['ArrowLength'] == arrowLength,'Spine'].to_frame()
        y2=dataset.loc[dataset['ArrowLength'] == arrowLength, str(poundageType)+'Poundage'].to_frame()
        y2.columns = ["Poundage"]
        df3 = pd.concat([x,y2],axis=1)
        df4 = df4.append(df3)

        
    df4 = df4.reset_index(drop=True)

    reg.fit(df4['Spine'].reshape(len(df4),1),df4['Poundage'].reshape(len(df4),1))

    x_test = pd.DataFrame(np.linspace(150,400,30).reshape(30,1))
    y_pred = pd.DataFrame(reg.predict(x_test))

    df2 = pd.concat([x_test,y_pred],axis=1)
    df2.columns = ["Spine", 'Poundage']

    return df4, df2, reg.coef_, reg.intercept_

#Function to calculate the FOC, Front of Center Percentage based on chosen arrow parameters over a poundage range
def calculateFOCdf(df,
    chosenSpine,
    chosenArrowGPI,
    chosenArrowLength,
    chosenNockThroatAdder,
    chosenNockWeight,
    chosenArrowWrapWeight,
    chosenArrowWrapLength,
    chosenFletchDistanceFromShaftEnd,
    chosenFletchNumber,
    chosenFletchWeight,
    chosenFletchLength
    ):

    totalFletchWeight = chosenFletchNumber * chosenFletchWeight
    totalShaftWeight = chosenArrowGPI * chosenArrowLength

    centroidNock = chosenNockThroatAdder
    centroidArrowWrap = chosenNockThroatAdder + chosenArrowWrapLength/2
    centroidFletch = chosenFletchDistanceFromShaftEnd + chosenFletchLength/3
    centroidShaft = chosenNockThroatAdder + chosenArrowLength/2
    centroidPointWeight = chosenNockThroatAdder + chosenArrowLength
    
    arrowLengthTotal = chosenArrowLength + chosenNockThroatAdder

    df2 = df.copy()
    
    for idx, row in df2.iterrows():
        totalArrowMass = chosenNockWeight + chosenArrowWrapWeight + totalFletchWeight + totalShaftWeight + df2.loc[idx]
        centerOfMassLength = (chosenNockWeight * chosenNockThroatAdder + chosenArrowWrapLength * centroidArrowWrap + totalFletchWeight * centroidFletch + totalShaftWeight * centroidShaft + df2.loc[idx] * centroidPointWeight)/totalArrowMass
        calculatedFOC = (100 *(centerOfMassLength - arrowLengthTotal/2) )/arrowLengthTotal
        df2.loc[idx] = round(calculatedFOC,2)
        
    return df2

In [8]:
#Code to derive the linear relationship between Bow IBO and effective poundage for Spine Charts from manufacturers

d = {'IBO': [300,320,345,350,300,330,315,330,300,330], 'Poundage': [-5,0,5,10,-5,0,-5,0,-5,0]}
dfIBOvsPoundage = pd.DataFrame(data=d)

x=dfIBOvsPoundage['IBO'].to_frame()
y=dfIBOvsPoundage['Poundage'].to_frame()

df = pd.concat([x,y],axis=1)

regIBO = LinearRegression()

regIBO.fit(x,y)

x_test = pd.DataFrame(np.linspace(300,350,30).reshape(30,1))
y_pred = pd.DataFrame(regIBO.predict(x_test))

df2 = pd.concat([x_test,y_pred],axis=1)
df2.columns = ["IBO", "Poundage"]

p2 = figure(title="IBO vs Poundage", x_axis_label="IBO", y_axis_label="Poundage Adder",plot_height=500, plot_width=500, y_range=(-10,15))
p2.circle(df['IBO'], df['Poundage'])
p2.line(df2['IBO'],df2['Poundage'], legend_label='LinearFit '+str(regIBO.coef_).replace("[", "").replace("]", "")+"* IBO +"+str(regIBO.intercept_).replace("[", "").replace("]", ""), color='red')
# show(p2)

print("IBO to Poundage Slope is ")
print(regIBO.coef_)

print("IBO to Poundage Intercept is ")
print(regIBO.intercept_)

#IBO to Poundage Slope is 0.25249169
#IBO to Poundage Intercept is -81.80232558

IBO to Poundage Slope is 
[[0.25249169]]
IBO to Poundage Intercept is 
[-81.80232558]


In [9]:

#This section visualizes the spread of GPIs for a given Spine
#Feel free to check out the GPI by arrow and Spine in the github repository
#Tip: If you want high FOC and don't care about price or reliability, pick the lowest GPI for a given Spine

sourceArrowGPIs = ColumnDataSource(data=dict(
    x=datasetArrowGPIs['Spine'],
    y=datasetArrowGPIs['GPI'],
    desc=datasetArrowGPIs['Arrow Name'],
))


TOOLTIPS = [
    ("(Spine,GPI)", "($x, $y)"),
    ("Arrow Name", "@desc"),
]

p3 = figure(title="Arrow Spine vs GPI Map", x_axis_label="Spine", y_axis_label="GPI",plot_height=500, plot_width=500, y_range=(0,20),tooltips=TOOLTIPS)
# p3.circle(datasetArrowGPIs['Spine'], datasetArrowGPIs['GPI'], size=7)
p3.circle('x', 'y', size=7, source=sourceArrowGPIs)

show(p3)

In [10]:


#Example set up for my Mach34 with an Sirius Orion Arrow
chosenSpine = 200
chosenArrowGPI = 10.7
chosenPoundage = 71
chosenIBO = 335
chosenArrowLength = 28.25
chosenNockThroatAdder = 0.5
chosenNockWeight = 6
chosenArrowWrapWeight = 0
chosenArrowWrapLength = 4
chosenFletchDistanceFromShaftEnd = 0.75
chosenFletchNumber = 4
chosenFletchWeight = 5
chosenFletchLength = 2.25
chosenFletchHeight = 0.465
chosenDrawLength = 29
chosenCoefDrag = 2
chosenArrowDiam = 0.166
chosenFletchOffset = 3


In [11]:


#Determining the Linear Regression parameters for the nominal, lower bound, and upper bounds
poundageTypes = ['Nominal', 'LowerBound', 'UpperBound']
d = {'Nominal': [0.000,0.000,0.000,0.000], 'LowerBound': [0.000,0.000,0.000,0.000], 'UpperBound': [0.000,0.000,0.000,0.000]}
poundageRegValues = pd.DataFrame(data=d)
poundageRegValues = pd.DataFrame(d, index=['slopeSlope', 'slopeIntercept', 'intSlope', 'intIntercept'])

#Determining the optimal point weights for different arrow manufacturers and the aggregate
d2 = {'Nominal': [0,0,0,0,0,0], 'LowerBound': [0,0,0,0,0,0], 'UpperBound': [0,0,0,0,0,0]}
estimatedPointWeights = pd.DataFrame(data=d2)
estimatedPointWeights = pd.DataFrame(d2, index=['Aggregate', 'BlackEagle', 'Easton', 'Goldtip','Sirius', 'Victory'])

for poundageType in poundageTypes:
    reset_output()

    #Plot Spine vs Effective Nominal Poundage by Arrow Length and Shaft Brand
    plots2 = []

    numLines = 50
    colors = itertools.cycle(Set2[5])
    # colors = itertools.cycle(inferno(numLines))

    for arrowLengths in range(26, 32):
        df, df2, coef, intercept = performLinRegbyArrowLength(dataset, arrowLengths, poundageType)

        p = figure(title="Arrow Length " + str(arrowLengths) + '"', x_axis_label="Spine", y_axis_label="Effective Poundage",plot_height=500, plot_width=500, y_range=(30,150))
        p.line(df2['Spine'],df2[str(poundageType)+'Poundage'], legend_label='Aggregate LinearFit '+str(coef).replace("[", "").replace("]", "")+"* ArrowLength +"+str(intercept).replace("[", "").replace("]", ""), color='red')
        for brand in dataset.Shaft.unique():
            df3, df4, coef2, intercept2 = performLinRegbyArrowLengthAndBrand(dataset, arrowLengths, brand, poundageType)
            colorCurrent=next(colors)
            p.circle(df3['Spine'], df3[str(poundageType)+'Poundage'], color=colorCurrent) #legend_label=brand + ' Spine-NominalPoundage', 
            p.line(df4['Spine'],df4[str(poundageType)+'Poundage'], legend_label=brand + ' LinearFit '+str(coef2).replace("[", "").replace("]", "")+"* ArrowLength +"+str(intercept2).replace("[", "").replace("]", ""), color=colorCurrent, line_dash='dashed')
        plots2.append(p)  

    #show(row(*plots2)) #Uncomment to show plots



    #Plot Spine vs Effective Nominal Poundage by Arrow Length
    plots = []
    d = {'ArrowLength': [], 'Slope': [],'Intercept': []}
    arrowRegs = pd.DataFrame(data=d)

    for arrowLengths in range(26, 32):
        df, df2, coef, intercept = performLinRegbyArrowLength(dataset, arrowLengths, poundageType)

        coef = coef.round(2)
        intercept = intercept.round(2)

        p = figure(title="Arrow Length " + str(arrowLengths) + '"', x_axis_label="Spine", y_axis_label="Effective Poundage",plot_height=500, plot_width=500, y_range=(30,100))
        p.circle(df['Spine'], df[str(poundageType)+'Poundage'], legend_label='Shaft-Spine '+str(poundageType)+'Poundage')
        p.line(df2['Spine'],df2[str(poundageType)+'Poundage'], legend_label='LinearFit '+str(coef).replace("[", "").replace("]", "")+"* ArrowLength +"+str(intercept).replace("[", "").replace("]", ""), color='red')
        plots.append(p)
        arrowRegs = arrowRegs.append({'ArrowLength': arrowLengths, 'Slope' : coef[0][0], 'Intercept' : intercept[0]}, ignore_index=True)    

    #show(row(*plots)) #Uncomment to show plots



    #Linear regression of y-Intercepts as a function of Arrow Length
    reg = LinearRegression()
    x=arrowRegs['ArrowLength'].to_frame()
    y=arrowRegs['Intercept'].to_frame()
    df = pd.concat([x,y],axis=1)
    reg.fit(x,y )
    arrowLengthRegIntSlope = reg.coef_.round(3)
    arrowLengthRegIntIntercept = reg.intercept_.round(3)

    #Plot y-Intercepts as a function of Arrow Length
    x_test2 = pd.DataFrame(np.linspace(25,32,30).reshape(30,1))
    y_pred2 = pd.DataFrame(reg.predict(x_test2))

    df5 = pd.concat([x_test2,y_pred2],axis=1)
    df5.columns = ["ArrowLength", "Intercept"]

    q = figure(title="Spine vs Poundage by Arrow Length - y-Intercepts for"+str(poundageType)+'Poundage', x_axis_label="Arrow Length", y_axis_label="ArrowRegs Intercept",plot_height=600, plot_width=600)
    q.circle(arrowRegs['ArrowLength'], arrowRegs['Intercept'], color='red')
    q.line(df5['ArrowLength'],df5['Intercept'], legend_label='LinearFit '+str(arrowLengthRegIntSlope).replace("[", "").replace("]", "")+"* ArrowLength +"+str(arrowLengthRegIntIntercept).replace("[", "").replace("]", ""), color='red')
    #show(q)

    #Equation for Optimal Poundage with Static Slope
    arrowRegSlope = sum(arrowRegs['Slope'])/len(arrowRegs['Slope'])
    round(arrowRegSlope,3)

    print("Equation for Optimal Poundage with Static Slope for "+str(poundageType)+'Poundage')
    print("Effective Poundage = " + str(arrowRegSlope).replace("[", "").replace("]", "") + "*Spine + "
          + str(arrowLengthRegIntSlope).replace("[", "").replace("]", "") 
          + "*Arrow Length +" + str(arrowLengthRegIntIntercept).replace("[", "").replace("]", ""))
    print("Effective Poundage = Poundage + 5*(Point Weight - 150gr)/25gr + 0.252*IBO - 81.8")
    print("Point Weight = 150gr + 25gr/5 * (-0.252 * IBO + 81.8 -Poundage "+ str(arrowRegSlope).replace("[", "").replace("]", "") + "*Spine + "
          + str(arrowLengthRegIntSlope).replace("[", "").replace("]", "") 
          + "*Arrow Length +" + str(arrowLengthRegIntIntercept).replace("[", "").replace("]", "") + ")")
    print()
    

    #Linear Regssion of Slopes as a function of Arrow Length
    reg2 = LinearRegression()
    x2=arrowRegs['ArrowLength'].to_frame()
    y2=arrowRegs['Slope'].to_frame()
    df2 = pd.concat([x2,y2],axis=1)
    reg2.fit(x2,y2)
    arrowLengthRegSlopeSlope = reg2.coef_.round(3)
    arrowLengthRegSlopeIntercept = reg2.intercept_.round(3)
    
    
    #Store Regression Values
    poundageRegValues[poundageType]['slopeSlope'] = arrowLengthRegSlopeSlope
    poundageRegValues[poundageType]['slopeIntercept'] = arrowLengthRegSlopeIntercept
    poundageRegValues[poundageType]['intSlope'] = arrowLengthRegIntSlope
    poundageRegValues[poundageType]['intIntercept'] = arrowLengthRegIntIntercept
    
    aggregateRegValuesSlopeSlope = poundageRegValues['Nominal']['slopeSlope']
    aggregateRegValuesSlopeIntercept = poundageRegValues['Nominal']['slopeIntercept']
    aggregateRegValuesIntSlope = poundageRegValues['Nominal']['intSlope']
    aggregateRegValuesIntIntercept = poundageRegValues['Nominal']['intIntercept']

    #Plot Slopes as a function of Arrow Length
    x_test3 = pd.DataFrame(np.linspace(25,32,30).reshape(30,1))
    y_pred3 = pd.DataFrame(reg2.predict(x_test3))

    df6 = pd.concat([x_test3,y_pred3],axis=1)
    df6.columns = ["ArrowLength", "Slope"]

    r = figure(title="Spine vs Poundage by Arrow Length - Slopes for"+str(poundageType)+'Poundage', x_axis_label="Arrow Length", y_axis_label="ArrowRegs Slopes",plot_height=600, plot_width=600)
    r.circle(arrowRegs['ArrowLength'], arrowRegs['Slope'], color='red')
    r.line(df6['ArrowLength'],df6['Slope'], legend_label='LinearFit '+str(arrowLengthRegSlopeSlope).replace("[", "").replace("]", "")+"* ArrowLength +"+str(arrowLengthRegSlopeIntercept).replace("[", "").replace("]", ""), color='red')
    #show(r)

    #Equation for Optimal Poundage with Variable Slope
    print("Equation for Optimal Poundage with Variable Slope for "+str(poundageType)+'Poundage')
    print("Effective Poundage = (" + str(arrowLengthRegSlopeSlope).replace("[", "").replace("]", "") 
          + "*Arrow Length +" + str(arrowLengthRegSlopeIntercept).replace("[", "").replace("]", "")
          + ")*Spine + "
          + str(arrowLengthRegIntSlope).replace("[", "").replace("]", "") 
          + "*Arrow Length +" + str(arrowLengthRegIntIntercept).replace("[", "").replace("]", ""))
    print("Effective Poundage = Poundage + 5*(Point Weight - 150gr)/25gr + 0.252*IBO - 81.8")
    print("Point Weight = 150gr + 25gr/5 * (-0.252 * IBO + 81.8 -Poundage + ("+ str(arrowLengthRegSlopeSlope).replace("[", "").replace("]", "") 
          + "*Arrow Length +" + str(arrowLengthRegSlopeIntercept).replace("[", "").replace("]", "")
          + ")*Spine + "
          + str(arrowLengthRegIntSlope).replace("[", "").replace("]", "") 
          + "*Arrow Length +" + str(arrowLengthRegIntIntercept).replace("[", "").replace("]", "") + ")")
    print()



for poundageType in poundageTypes:
    estimatedPointWeights[poundageType]['Aggregate'] = 150+25/5 * (-0.252 * chosenIBO + 81.8 -chosenPoundage + (poundageRegValues[poundageType]['slopeSlope']*chosenArrowLength 
                    + poundageRegValues[poundageType]['slopeIntercept']) * chosenSpine 
                    + poundageRegValues[poundageType]['intSlope']*chosenArrowLength 
                    + poundageRegValues[poundageType]['intIntercept'])  
    
    
for brand in dataset.Shaft.unique():
    d = {'Nominal': [0.000,0.000,0.000,0.000], 'LowerBound': [0.000,0.000,0.000,0.000], 'UpperBound': [0.000,0.000,0.000,0.000]}
    poundageRegValues = pd.DataFrame(data=d)
    poundageRegValues = pd.DataFrame(d, index=['slopeSlope', 'slopeIntercept', 'intSlope', 'intIntercept'])
    
    for poundageType in poundageTypes:
        d3 = {'ArrowLength': [], 'Slope': [],'Intercept': []}
        arrowRegs = pd.DataFrame(data=d3)

        for arrowLengths in range(26, 32):
            df3, df4, coef2, intercept2 = performLinRegbyArrowLengthAndBrand(dataset, arrowLengths, brand, poundageType)
            arrowRegs = arrowRegs.append({'ArrowLength': arrowLengths, 'Slope' : coef2[0][0], 'Intercept' : intercept2[0]}, ignore_index=True)    

        #Linear Regssion of y-Intercepts as a function of Arrow Length
        reg = LinearRegression()
        x=arrowRegs['ArrowLength'].to_frame()
        y=arrowRegs['Intercept'].to_frame()
        df = pd.concat([x,y],axis=1)
        reg.fit(x,y )
        arrowLengthRegIntSlope = reg.coef_.round(3)
        arrowLengthRegIntIntercept = reg.intercept_.round(3)

        #Linear Regssion of Slopes as a function of Arrow Length
        reg2 = LinearRegression()
        x2=arrowRegs['ArrowLength'].to_frame()
        y2=arrowRegs['Slope'].to_frame()
        df2 = pd.concat([x2,y2],axis=1)
        reg2.fit(x2,y2)
        arrowLengthRegSlopeSlope = reg2.coef_.round(3)
        arrowLengthRegSlopeIntercept = reg2.intercept_.round(3)


        #Store Regression Values
        poundageRegValues[poundageType]['slopeSlope'] = arrowLengthRegSlopeSlope
        poundageRegValues[poundageType]['slopeIntercept'] = arrowLengthRegSlopeIntercept
        poundageRegValues[poundageType]['intSlope'] = arrowLengthRegIntSlope
        poundageRegValues[poundageType]['intIntercept'] = arrowLengthRegIntIntercept
        
        """
        Nominal Starting Point Weight is 150 grains
        Assume Relationship between Nominal Starting Point Weight and Effective Poundage is 25:5 (From manufacturers)
        IBO to Poundage Slope is 0.252
        IBO to Poundage Intercept is -81.8
        """
        
        estimatedPointWeights[poundageType][brand] = 150+25/5 * (-0.252 * chosenIBO + 81.8-chosenPoundage 
                        + (poundageRegValues[poundageType]['slopeSlope']*chosenArrowLength 
                        + poundageRegValues[poundageType]['slopeIntercept']) * chosenSpine 
                        + poundageRegValues[poundageType]['intSlope']*chosenArrowLength 
                        + poundageRegValues[poundageType]['intIntercept'])
        



Equation for Optimal Poundage with Static Slope for NominalPoundage
Effective Poundage = -0.20666666666666667*Spine + -3.885*Arrow Length +237.637
Effective Poundage = Poundage + 5*(Point Weight - 150gr)/25gr + 0.252*IBO - 81.8
Point Weight = 150gr + 25gr/5 * (-0.252 * IBO + 81.8 -Poundage -0.20666666666666667*Spine + -3.885*Arrow Length +237.637)

Equation for Optimal Poundage with Variable Slope for NominalPoundage
Effective Poundage = (-0.001*Arrow Length +-0.174)*Spine + -3.885*Arrow Length +237.637
Effective Poundage = Poundage + 5*(Point Weight - 150gr)/25gr + 0.252*IBO - 81.8
Point Weight = 150gr + 25gr/5 * (-0.252 * IBO + 81.8 -Poundage + (-0.001*Arrow Length +-0.174)*Spine + -3.885*Arrow Length +237.637)

Equation for Optimal Poundage with Static Slope for LowerBoundPoundage
Effective Poundage = -0.21333333333333335*Spine + -5.693*Arrow Length +286.985
Effective Poundage = Poundage + 5*(Point Weight - 150gr)/25gr + 0.252*IBO - 81.8
Point Weight = 150gr + 25gr/5 * (-0.252 * IBO

In [12]:
print(estimatedPointWeights) #Display optimal point weights by brand or the aggregate
    
dfCalculatedFOC = calculateFOCdf(estimatedPointWeights,
    chosenSpine,
    chosenArrowGPI,
    chosenArrowLength,
    chosenNockThroatAdder,
    chosenNockWeight,
    chosenArrowWrapWeight,
    chosenArrowWrapLength,
    chosenFletchDistanceFromShaftEnd,
    chosenFletchNumber,
    chosenFletchWeight,
    chosenFletchLength
    )

print(dfCalculatedFOC) #Display FOC with the optimal point weights by brand or the aggregate



            Nominal  LowerBound  UpperBound
Aggregate       219         193         233
BlackEagle      185         183         216
Easton          240         216         265
Goldtip         222         212         232
Sirius          216         191         212
Victory         272         227         290
            Nominal  LowerBound  UpperBound
Aggregate     18.39       16.81       19.17
BlackEagle    16.29       16.16       18.21
Easton        19.55       18.21       20.84
Goldtip       18.56       17.98       19.12
Sirius        18.21       16.68       17.98
Victory       21.18       18.84       22.02


In [13]:
print(poundageRegValues)
print(aggregateRegValuesSlopeSlope)
print(aggregateRegValuesSlopeIntercept)
print(aggregateRegValuesIntSlope)
print(aggregateRegValuesIntIntercept)

"""
After all the above linear regression work, we'll use the following linear
regression values from now on.

Aggregate Linear Regression Values
ggregateRegValuesSlopeSlope = -0.001
aggregateRegValuesSlopeIntercept = -0.174
aggregateRegValuesIntSlope = -3.885
aggregateRegValuesIntIntercept = 237.637

Nominal Starting Point Weight is 150 grains
Assume Relationship between Nominal Starting Point Weight and Effective Poundage is 25:5 (From manufacturers)
IBO to Poundage Slope is 0.252
IBO to Poundage Intercept is -81.8

So the Optimal Point Weight will be calulated by the following equation:

calcOpPointWeight = 150+25/5 * (-0.252 * chosenIBO + 81.8 -calcPoundage + 
                    (aggregateRegValuesSlopeSlope*chosenArrowLength 
                    + aggregateRegValuesSlopeIntercept) * chosenSpine 
                    + aggregateRegValuesIntSlope*chosenArrowLength 
                    + aggregateRegValuesIntIntercept)  

Will also be using a default Drag Coefficient of 2, but feel free to play around with it.
Per the whitepaper: Meyer, H.O. (2015) Applications of Physics to Archery

"""



                Nominal  LowerBound  UpperBound
slopeSlope       -0.010      -0.001      -0.020
slopeIntercept    0.075      -0.178       0.329
intSlope          0.888      -2.428       4.204
intIntercept    114.498     198.870      30.125
-0.001
-0.174
-3.885
237.637


"\nAfter all the above linear regression work, we'll use the following linear\nregression values from now on.\n\nAggregate Linear Regression Values\nggregateRegValuesSlopeSlope = -0.001\naggregateRegValuesSlopeIntercept = -0.174\naggregateRegValuesIntSlope = -3.885\naggregateRegValuesIntIntercept = 237.637\n\nNominal Starting Point Weight is 150 grains\nAssume Relationship between Nominal Starting Point Weight and Effective Poundage is 25:5 (From manufacturers)\nIBO to Poundage Slope is 0.252\nIBO to Poundage Intercept is -81.8\n\nSo the Optimal Point Weight will be calulated by the following equation:\n\ncalcOpPointWeight = 150+25/5 * (-0.252 * chosenIBO + 81.8 -calcPoundage + \n                    (aggregateRegValuesSlopeSlope*chosenArrowLength \n                    + aggregateRegValuesSlopeIntercept) * chosenSpine \n                    + aggregateRegValuesIntSlope*chosenArrowLength \n                    + aggregateRegValuesIntIntercept)  \n\nWill also be using a default Drag Coeffic

In [14]:
#Side by Side comparison


#Poundage vs FOC

output_notebook()

# Set up data
aggregateRegValuesSlopeSlope = -0.001
aggregateRegValuesSlopeIntercept = -0.174
aggregateRegValuesIntSlope = -3.885
aggregateRegValuesIntIntercept = 237.637


#FOC Calculation Parameters

#Sirius Orion with Mach 34
chosenSpine = 200
chosenArrowGPI = 10.7
chosenPoundage = 71
chosenIBO = 335
chosenArrowLength = 28.25
chosenNockThroatAdder = 0.5
chosenNockWeight = 6
chosenArrowWrapWeight = 0
chosenArrowWrapLength = 4
chosenFletchDistanceFromShaftEnd = 0.75
chosenFletchNumber = 4
chosenFletchWeight = 5
chosenFletchLength = 2.25
chosenFletchHeight = 0.465
chosenDrawLength = 29
chosenCoefDrag = 2
chosenArrowDiam = 0.166
chosenFletchOffset = 3


#Victory RIP XV with Mach 34
chosenSpine2 = 300
chosenArrowGPI2 = 7.1
chosenPoundage2 = 71
chosenIBO2 = 335
chosenArrowLength2 = 28.25
chosenNockThroatAdder2 = 0.5
chosenNockWeight2 = 6
chosenArrowWrapWeight2 = 0
chosenArrowWrapLength2 = 4
chosenFletchDistanceFromShaftEnd2 = 0.75
chosenFletchNumber2 = 4
chosenFletchWeight2 = 5
chosenFletchLength2 = 2.25
chosenFletchHeight2 = 0.465
chosenDrawLength2 = 29
chosenCoefDrag2 = 2
chosenArrowDiam2 = 0.166
chosenFletchOffset2 = 3


def calculate_speed(initial_velocity, area_cross_section ,coefficient_drag, arrow_mass, distance):
    '''
    returns an array of different velocities given different initial_velocity
    '''

    # Constants
    air_density = 0.0752  # lb/ft^3
    if isinstance(initial_velocity, float):
        # Calculate the velocity at the given distance
        runtime = 3
        time_of_flight = 0
        distance_traveled = 0
        acceleration = lambda v: -0.5 * air_density * area_cross_section * coefficient_drag * v**2 / arrow_mass
        velocity = initial_velocity
        dt = 0.001  # time step size
        for t in range(int(runtime/dt)):
            velocity += acceleration(velocity) * dt
            time_of_flight = time_of_flight + dt
            distance_traveled = distance_traveled + velocity * dt
            if distance_traveled >= distance:
                break
        
        return velocity
        
    v_list = []
    for i in range(len(initial_velocity)):
        iv = initial_velocity[i]
        # Calculate the velocity at the given distance
        runtime = 3
        time_of_flight = 0
        distance_traveled = 0
        acceleration = lambda v: -0.5 * air_density * area_cross_section * coefficient_drag * v**2 / arrow_mass[i]
        velocity = iv
        dt = 0.001  # time step size
        for t in range(int(runtime/dt)):
            velocity += acceleration(velocity) * dt
            time_of_flight = time_of_flight + dt
            distance_traveled = distance_traveled + velocity * dt
            if distance_traveled >= distance:
                break
        v_list.append(velocity)

    # Return the velocity in feet per second
    return np.array(v_list)

def calculate_time(initial_velocity, area_cross_section ,coefficient_drag, arrow_mass, distance):
    '''
    returns an array of different time of flights given different initial_velocity
    '''

    # Constants
    air_density = 0.0752  # lb/ft^3
    if isinstance(initial_velocity, float):
        # Calculate the velocity at the given distance
        runtime = 3
        time_of_flight = 0
        distance_traveled = 0
        acceleration = lambda v: -0.5 * air_density * area_cross_section * coefficient_drag * v**2 / arrow_mass
        velocity = initial_velocity
        dt = 0.001  # time step size
        for t in range(int(runtime/dt)):
            velocity += acceleration(velocity) * dt
            time_of_flight = time_of_flight + dt
            distance_traveled = distance_traveled + velocity * dt
            if distance_traveled >= distance:
                break
        
        return time_of_flight
        
    t_list = []
    for i in range(len(initial_velocity)):
        iv = initial_velocity[i]
        # Calculate the velocity at the given distance
        runtime = 3
        time_of_flight = 0
        distance_traveled = 0
        acceleration = lambda v: -0.5 * air_density * area_cross_section * coefficient_drag * v**2 / arrow_mass[i]
        velocity = iv
        dt = 0.001  # time step size
        for t in range(int(runtime/dt)):
            velocity += acceleration(velocity) * dt
            time_of_flight = time_of_flight + dt
            distance_traveled = distance_traveled + velocity * dt
            if distance_traveled >= distance:
                break
        t_list.append(time_of_flight)

    # Return the velocity in feet per second
    return np.array(t_list)

#1st Arrow-Box Configuration

#Poundage [lbs]
calcPoundage = np.linspace(30, 90, 30)

#optimalPointWeight [gr]
calcOpPointWeight = 150+25/5 * (-0.252 * chosenIBO + 81.8 -calcPoundage + (aggregateRegValuesSlopeSlope*chosenArrowLength 
                    + aggregateRegValuesSlopeIntercept) * chosenSpine 
                    + aggregateRegValuesIntSlope*chosenArrowLength 
                    + aggregateRegValuesIntIntercept)  
sourceOpPointWeight = ColumnDataSource(data=dict(x=calcPoundage, y=calcOpPointWeight))

#totalArrowMass [gr]
calcTotalArrowMass = chosenNockWeight + chosenArrowWrapWeight + chosenFletchNumber * chosenFletchWeight + chosenArrowGPI * chosenArrowLength + calcOpPointWeight
sourceTotalArrowMass = ColumnDataSource(data=dict(x=calcPoundage, y=calcTotalArrowMass))

#FOC [%]      
calcFOC = (100 *((chosenNockWeight * chosenNockThroatAdder + chosenArrowWrapLength * (chosenNockThroatAdder + chosenArrowWrapLength/2) 
            + (chosenFletchNumber * chosenFletchWeight) * (chosenFletchDistanceFromShaftEnd + chosenFletchLength/3) 
            + (chosenArrowGPI * chosenArrowLength) * (chosenNockThroatAdder + chosenArrowLength/2) 
            + calcOpPointWeight * (chosenNockThroatAdder + chosenArrowLength))
            /(calcTotalArrowMass) 
            - (chosenArrowLength + chosenNockThroatAdder)/2))/(chosenArrowLength + chosenNockThroatAdder)     
sourceFOC = ColumnDataSource(data=dict(x=calcPoundage, y=calcFOC))

#Nominal Kinetic Energy [J]
calcKENominal = (0.5*((350/15.43)/1000)*((chosenIBO-10*(30-chosenDrawLength)-2*(70-calcPoundage))*0.3048)**2)


#FPS [f/s]
calcMPS = ((calcKENominal*2)/((calcTotalArrowMass/15.43)/1000))**0.5
calcFPS = (((calcKENominal*2)/((calcTotalArrowMass/15.43)/1000))**0.5)/0.3048
sourceFPS = ColumnDataSource(data=dict(x=calcPoundage, y=calcFPS))

#Cross Sectional area of arrow
area_cross_section = np.pi*((chosenArrowDiam/12)/2)**2 + chosenFletchNumber * 0.5 * chosenFletchLength/12 * chosenFletchHeight/12 * chosenFletchOffset/90  # ft^2

#Velocity [fps]
calcFPS20yd = calculate_speed(calcFPS, area_cross_section, chosenCoefDrag, calcTotalArrowMass/7000, 60)
calcFPS40yd = calculate_speed(calcFPS, area_cross_section, chosenCoefDrag, calcTotalArrowMass/7000, 120)
calcFPS60yd = calculate_speed(calcFPS, area_cross_section, chosenCoefDrag, calcTotalArrowMass/7000, 180)
sourceFPS20yd = ColumnDataSource(data=dict(x=calcPoundage, y=calcFPS20yd))
sourceFPS40yd = ColumnDataSource(data=dict(x=calcPoundage, y=calcFPS40yd))
sourceFPS60yd = ColumnDataSource(data=dict(x=calcPoundage, y=calcFPS60yd))

#Time of Flight [s]
calcTOF20yd = calculate_time(calcFPS, area_cross_section, chosenCoefDrag, calcTotalArrowMass/7000, 60)
calcTOF40yd = calculate_time(calcFPS, area_cross_section, chosenCoefDrag, calcTotalArrowMass/7000, 120)
calcTOF60yd = calculate_time(calcFPS, area_cross_section, chosenCoefDrag, calcTotalArrowMass/7000, 180)
sourceTOF20yd = ColumnDataSource(data=dict(x=calcPoundage, y=calcTOF20yd))
sourceTOF40yd = ColumnDataSource(data=dict(x=calcPoundage, y=calcTOF40yd))
sourceTOF60yd = ColumnDataSource(data=dict(x=calcPoundage, y=calcTOF60yd))
  

#Kinetic Energy [J]
calcKE = 0.5*((calcMPS**2)*((calcTotalArrowMass/15.43)/1000))
sourceKE = ColumnDataSource(data=dict(x=calcPoundage, y=calcKE))
calcKE20yd = 0.5*(((calcFPS20yd*0.3048)**2)*((calcTotalArrowMass/15.43)/1000))
calcKE40yd = 0.5*(((calcFPS40yd*0.3048)**2)*((calcTotalArrowMass/15.43)/1000))
calcKE60yd = 0.5*(((calcFPS60yd*0.3048)**2)*((calcTotalArrowMass/15.43)/1000))
sourceKE20yd = ColumnDataSource(data=dict(x=calcPoundage, y=calcKE20yd))
sourceKE40yd = ColumnDataSource(data=dict(x=calcPoundage, y=calcKE40yd))
sourceKE60yd = ColumnDataSource(data=dict(x=calcPoundage, y=calcKE60yd))

#Momentum [kg*m/s]
calcMomentum = ((calcTotalArrowMass/15.43)/1000)*calcMPS
sourceMomentum = ColumnDataSource(data=dict(x=calcPoundage, y=calcMomentum))
calcMomentum20yd = ((calcTotalArrowMass/15.43)/1000)*calcFPS20yd*0.3048
calcMomentum40yd = ((calcTotalArrowMass/15.43)/1000)*calcFPS40yd*0.3048
calcMomentum60yd = ((calcTotalArrowMass/15.43)/1000)*calcFPS60yd*0.3048
sourceMomentum20yd = ColumnDataSource(data=dict(x=calcPoundage, y=calcMomentum20yd))
sourceMomentum40yd = ColumnDataSource(data=dict(x=calcPoundage, y=calcMomentum40yd))
sourceMomentum60yd = ColumnDataSource(data=dict(x=calcPoundage, y=calcMomentum60yd))

calcKEdotMomentum = calcKE*calcMomentum
sourceKEdotMomentum = ColumnDataSource(data=dict(x=calcPoundage, y=calcKEdotMomentum))
calcKEdotMomentum20yd = calcKE20yd*calcMomentum20yd
calcKEdotMomentum40yd = calcKE40yd*calcMomentum40yd
calcKEdotMomentum60yd = calcKE60yd*calcMomentum60yd
sourceKEdotMomentum20yd = ColumnDataSource(data=dict(x=calcPoundage, y=calcKEdotMomentum20yd))
sourceKEdotMomentum40yd = ColumnDataSource(data=dict(x=calcPoundage, y=calcKEdotMomentum40yd))
sourceKEdotMomentum60yd = ColumnDataSource(data=dict(x=calcPoundage, y=calcKEdotMomentum60yd))

# Set up plot
plotOpPointWeight = figure(height=300, width=300, title="Poundage vs Optimal Point Weight [grains]",
              x_range=[30,90], y_range=[0, 500])
plotTotalArrowMass = figure(height=300, width=300, title="Poundage vs TotalArrowMass [grains]",
              x_range=[30,90], y_range=[250, 800])
plotFOC = figure(height=300, width=300, title="Poundage vs FOC [%]",
              x_range=[30,90], y_range=[0, 35])
plotKE = figure(height=300, width=300, title="Poundage vs Kinetic Energy [J]",
              x_range=[30,90], y_range=[0, 300])
plotFPS = figure(height=300, width=300, title="Poundage vs FPS",
              x_range=[30,90], y_range=[150, 500])
plotTOF = figure(height=300, width=300, title="Poundage vs Time of Flight [s]",
              x_range=[30,90], y_range=[0, 2])
plotMomentum = figure(height=300, width=300, title="Poundage vs Momentum [kg*m/s]",
              x_range=[30,90], y_range=[0, 5])
plotKEdotMomentum = figure(height=300, width=300, title="Poundage vs KE*Momentum",
              x_range=[30,90], y_range=[0, 1500])
plotKEdotMomentum = figure(height=300, width=300, title="Poundage vs KE*Momentum",
              x_range=[30,90], y_range=[0, 800])

selectedPoundage=chosenPoundage
selectedOpPointWeight=round(150+25/5 * (-0.252 * chosenIBO + 81.8 -selectedPoundage + (aggregateRegValuesSlopeSlope*chosenArrowLength + aggregateRegValuesSlopeIntercept) * chosenSpine + aggregateRegValuesIntSlope*chosenArrowLength + aggregateRegValuesIntIntercept),0)
selectedTotalArrowMass=round(chosenNockWeight + chosenArrowWrapWeight + chosenFletchNumber * chosenFletchWeight + chosenArrowGPI * chosenArrowLength + selectedOpPointWeight,0)
selectedFOC=round((100 *((chosenNockWeight * chosenNockThroatAdder + chosenArrowWrapLength * (chosenNockThroatAdder + chosenArrowWrapLength/2) + (chosenFletchNumber * chosenFletchWeight) * (chosenFletchDistanceFromShaftEnd + chosenFletchLength/3) + (chosenArrowGPI * chosenArrowLength) * (chosenNockThroatAdder + chosenArrowLength/2) + selectedOpPointWeight * (chosenNockThroatAdder + chosenArrowLength))/(chosenNockWeight + chosenArrowWrapWeight + chosenFletchNumber * chosenFletchWeight + chosenArrowGPI * chosenArrowLength + selectedOpPointWeight) - (chosenArrowLength + chosenNockThroatAdder)/2))/(chosenArrowLength + chosenNockThroatAdder),2)
selectedKENominal=round((0.5*((350/15.43)/1000)*((chosenIBO-10*(30-chosenDrawLength)-2*(70-selectedPoundage))*0.3048)**2),0)
selectedFPS=round(((((0.5*((350/15.43)/1000)*((chosenIBO-10*(30-chosenDrawLength)-2*(70-selectedPoundage))*0.3048)**2) * 2)/((selectedTotalArrowMass/15.43)/1000))**0.5)/0.3048,0)
selectedKE=round(0.5*(((selectedTotalArrowMass/15.43)/1000)*(((0.5*((350/15.43)/1000)*((chosenIBO-10*(30-chosenDrawLength)-2*(70-selectedPoundage))*0.3048)**2)*2)/((selectedTotalArrowMass/15.43)/1000))),0)
selectedMomentum=round(((selectedTotalArrowMass/15.43)/1000)*(((0.5*((350/15.43)/1000)*(((chosenIBO-10*(30-chosenDrawLength)-2*(70-selectedPoundage))*0.3048)**2)*2)/((selectedTotalArrowMass/15.43)/1000))**0.5),2)
selectedKEdotMomentum=round(selectedKE*selectedMomentum,0)
selectedFPS60yd=round(calculate_speed(selectedFPS, area_cross_section, chosenCoefDrag, selectedTotalArrowMass/7000, 180),0)
selectedTOF20yd = round(calculate_time(selectedFPS, area_cross_section, chosenCoefDrag, selectedTotalArrowMass/7000, 60),3)
selectedTOF60yd = round(calculate_time(selectedFPS, area_cross_section, chosenCoefDrag, selectedTotalArrowMass/7000, 180),3)
selectedKE60yd = round(0.5*(((selectedFPS60yd*0.3048)**2)*((selectedTotalArrowMass/15.43)/1000)),0)
selectedMomentum60yd = round(((selectedTotalArrowMass/15.43)/1000)*selectedFPS60yd*0.3048,2)
selectedKEdotMomentum60yd = round(selectedKE60yd*selectedMomentum60yd,0)


sourceChosenPoundage = ColumnDataSource(data=dict(selectedPoundage=[selectedPoundage],
                                                  selectedOpPointWeight=[selectedOpPointWeight],
                                                  selectedTotalArrowMass=[selectedTotalArrowMass],
                                                  selectedFOC=[selectedFOC],
                                                  selectedFPS=[selectedFPS],
                                                  selectedKE=[selectedKE],
                                                  selectedMomentum=[selectedMomentum],
                                                  selectedKEdotMomentum=[selectedKEdotMomentum],
                                                  selectedFPS60yd=[selectedFPS60yd],
                                                  selectedTOF20yd=[selectedTOF20yd],
                                                  selectedTOF60yd=[selectedTOF60yd],
                                                  selectedKE60yd=[selectedKE60yd],
                                                  selectedMomentum60yd=[selectedMomentum60yd],
                                                  selectedKEdotMomentum60yd=[selectedKEdotMomentum60yd]
                                                 ))



labelOpPointWeight = LabelSet(x='selectedPoundage', y='selectedOpPointWeight', text=str('selectedOpPointWeight'),
                  x_offset=-40, y_offset=5, source=sourceChosenPoundage, text_color ="blue")
labelTotalArrowMass = LabelSet(x='selectedPoundage', y='selectedTotalArrowMass', text=str('selectedTotalArrowMass'),
                  x_offset=-40, y_offset=5, source=sourceChosenPoundage, text_color ="blue")
labelFOC = LabelSet(x='selectedPoundage', y='selectedFOC', text=str('selectedFOC'),
                  x_offset=-40, y_offset=5, source=sourceChosenPoundage, text_color ="blue")
labelFPS = LabelSet(x='selectedPoundage', y='selectedFPS', text=str('selectedFPS'),
                  x_offset=-40, y_offset=5, source=sourceChosenPoundage, text_color ="blue")
labelKE = LabelSet(x='selectedPoundage', y='selectedKE', text=str('selectedKE'),
                  x_offset=-40, y_offset=5, source=sourceChosenPoundage, text_color ="blue")
labelMomentum = LabelSet(x='selectedPoundage', y='selectedMomentum', text=str('selectedMomentum'),
                  x_offset=-40, y_offset=5, source=sourceChosenPoundage, text_color ="blue")
labelKEdotMomentum = LabelSet(x='selectedPoundage', y='selectedKEdotMomentum', text=str('selectedKEdotMomentum'),
                  x_offset=-40, y_offset=5, source=sourceChosenPoundage, text_color ="blue")


labelFPS60yd = LabelSet(x='selectedPoundage', y='selectedFPS60yd', text=str('selectedFPS60yd'),
                  x_offset=-40, y_offset=5, source=sourceChosenPoundage, text_color ="blue")
labelTOF20yd = LabelSet(x='selectedPoundage', y='selectedTOF20yd', text=str('selectedTOF20yd'),
                  x_offset=-40, y_offset=5, source=sourceChosenPoundage, text_color ="blue")
labelTOF60yd = LabelSet(x='selectedPoundage', y='selectedTOF60yd', text=str('selectedTOF60yd'),
                  x_offset=-40, y_offset=5, source=sourceChosenPoundage, text_color ="blue")
labelKE60yd = LabelSet(x='selectedPoundage', y='selectedKE60yd', text=str('selectedKE60yd'),
                  x_offset=-40, y_offset=5, source=sourceChosenPoundage, text_color ="blue")
labelMomentum60yd = LabelSet(x='selectedPoundage', y='selectedMomentum60yd', text=str('selectedMomentum60yd'),
                  x_offset=-40, y_offset=5, source=sourceChosenPoundage, text_color ="blue")
labelKEdotMomentum60yd = LabelSet(x='selectedPoundage', y='selectedKEdotMomentum60yd', text=str('selectedKEdotMomentum60yd'),
                  x_offset=-40, y_offset=5, source=sourceChosenPoundage, text_color ="blue")


plotOpPointWeight.line('x', 'y', source=sourceOpPointWeight, line_width=3, line_alpha=0.6)
plotTotalArrowMass.line('x', 'y', source=sourceTotalArrowMass, line_width=3, line_alpha=0.6)
plotFOC.line('x', 'y', source=sourceFOC, line_width=3, line_alpha=0.6)
plotFPS.line('x', 'y', source=sourceFPS, line_width=3, line_alpha=0.6)
plotKE.line('x', 'y', source=sourceKE, line_width=3, line_alpha=0.6)
plotMomentum.line('x', 'y', source=sourceMomentum, line_width=3, line_alpha=0.6)
plotKEdotMomentum.line('x', 'y', source=sourceKEdotMomentum, line_width=3, line_alpha=0.6)
plotFPS.line('x', 'y', source=sourceFPS20yd, line_width=3, line_alpha=0.6, line_dash = "dashed")
plotFPS.line('x', 'y', source=sourceFPS40yd, line_width=3, line_alpha=0.6, line_dash = "dashdot")
plotFPS.line('x', 'y', source=sourceFPS60yd, line_width=3, line_alpha=0.6, line_dash = "dotted")
plotTOF.line('x', 'y', source=sourceTOF20yd, line_width=3, line_alpha=0.6, line_dash = "dashed")
plotTOF.line('x', 'y', source=sourceTOF40yd, line_width=3, line_alpha=0.6, line_dash = "dashdot")
plotTOF.line('x', 'y', source=sourceTOF60yd, line_width=3, line_alpha=0.6, line_dash = "dotted")
plotKE.line('x', 'y', source=sourceKE20yd, line_width=3, line_alpha=0.6, line_dash = "dashed")
plotMomentum.line('x', 'y', source=sourceMomentum20yd, line_width=3, line_alpha=0.6, line_dash = "dashed")
plotKEdotMomentum.line('x', 'y', source=sourceKEdotMomentum20yd, line_width=3, line_alpha=0.6, line_dash = "dashed")
plotKE.line('x', 'y', source=sourceKE40yd, line_width=3, line_alpha=0.6, line_dash = "dashdot")
plotMomentum.line('x', 'y', source=sourceMomentum40yd, line_width=3, line_alpha=0.6, line_dash = "dashdot")
plotKEdotMomentum.line('x', 'y', source=sourceKEdotMomentum40yd, line_width=3, line_alpha=0.6, line_dash = "dashdot")
plotKE.line('x', 'y', source=sourceKE60yd, line_width=3, line_alpha=0.6, line_dash = "dotted")
plotMomentum.line('x', 'y', source=sourceMomentum60yd, line_width=3, line_alpha=0.6, line_dash = "dotted")
plotKEdotMomentum.line('x', 'y', source=sourceKEdotMomentum60yd, line_width=3, line_alpha=0.6, line_dash = "dotted")

plotOpPointWeight.circle('selectedPoundage','selectedOpPointWeight', source=sourceChosenPoundage, line_color="yellow", size=12)
plotTotalArrowMass.circle('selectedPoundage','selectedTotalArrowMass', source=sourceChosenPoundage, line_color="yellow", size=12)
plotFOC.circle('selectedPoundage','selectedFOC', source=sourceChosenPoundage, line_color="yellow", size=12)
plotKE.circle('selectedPoundage','selectedKE', source=sourceChosenPoundage, line_color="yellow", size=12)
plotFPS.circle('selectedPoundage','selectedFPS', source=sourceChosenPoundage, line_color="yellow", size=12)
plotMomentum.circle('selectedPoundage','selectedMomentum', source=sourceChosenPoundage, line_color="yellow", size=12)
plotKEdotMomentum.circle('selectedPoundage','selectedKEdotMomentum', source=sourceChosenPoundage, line_color="yellow", size=12)
# plotFPS.circle('selectedPoundage','selectedFPS20yd', source=sourceChosenPoundage, line_color="yellow", size=12)
# plotFPS.circle('selectedPoundage','selectedFPS40yd', source=sourceChosenPoundage, line_color="yellow", size=12)
plotFPS.circle('selectedPoundage','selectedFPS60yd', source=sourceChosenPoundage, line_color="yellow", size=12)
plotTOF.circle('selectedPoundage','selectedTOF20yd', source=sourceChosenPoundage, line_color="yellow", size=12)
plotTOF.circle('selectedPoundage','selectedTOF60yd', source=sourceChosenPoundage, line_color="yellow", size=12)
plotKE.circle('selectedPoundage','selectedKE60yd', source=sourceChosenPoundage, line_color="yellow", size=12)
plotMomentum.circle('selectedPoundage','selectedMomentum60yd', source=sourceChosenPoundage, line_color="yellow", size=12)
plotKEdotMomentum.circle('selectedPoundage','selectedKEdotMomentum60yd', source=sourceChosenPoundage, line_color="yellow", size=12)

plotOpPointWeight.add_layout(labelOpPointWeight)
plotTotalArrowMass.add_layout(labelTotalArrowMass)
plotFOC.add_layout(labelFOC)
plotFPS.add_layout(labelFPS)
plotKE.add_layout(labelKE)
plotMomentum.add_layout(labelMomentum)
plotKEdotMomentum.add_layout(labelKEdotMomentum)
plotFPS.add_layout(labelFPS60yd)
plotTOF.add_layout(labelTOF20yd)
plotTOF.add_layout(labelTOF60yd)
plotKE.add_layout(labelKE60yd)
plotMomentum.add_layout(labelMomentum60yd)
plotKEdotMomentum.add_layout(labelKEdotMomentum60yd)


#KE Bands
#Kinetic energy vs animal
#Small Game - 25ft lbs... <34J
#Medium - Deer 25-41... 35-56J
#Large - Elk 41-65...  56-88
#Extreme - Moose 65+... >88
smallKE = np.linspace(0, 0, 30)
mediumKE = np.linspace(35, 35, 30)
highKE = np.linspace(55, 55, 30)
extremeKE = np.linspace(88, 88, 30)
topKE = np.linspace(300, 300, 30)

sourceKEband = ColumnDataSource({
        'base':calcPoundage,
        'small':smallKE,
        'medium':mediumKE,
        'high':highKE,
        'extreme':extremeKE,
        'top':topKE
        })

smallKEband = Band(base='base', lower='small', upper='medium', source=sourceKEband, fill_alpha=0.5, level='underlay', fill_color = "red")
mediumKEband = Band(base='base', lower='medium', upper='high', source=sourceKEband, fill_alpha=0.5, level='underlay', fill_color = "#90CAF9")
highKEband = Band(base='base', lower='high', upper='extreme', source=sourceKEband, fill_alpha=0.5, level='underlay', fill_color = "#42A5F5")
extremeKEband = Band(base='base', lower='extreme', upper='top', source=sourceKEband, fill_alpha=0.5, level='underlay', fill_color = "#1E88E5")


plotKE.add_layout(smallKEband)
plotKE.add_layout(mediumKEband)
plotKE.add_layout(highKEband)
plotKE.add_layout(extremeKEband)

#FOC Bands
bottomFOC = np.linspace(0, 0, 30)
normalFOC = np.linspace(12, 12, 30)
highFOC = np.linspace(19, 19, 30)
extremeFOC = np.linspace(30, 30, 30)
#ultraExtremeFOC = np.linspace(50, 30, 30)
topFOC = np.linspace(50, 50, 30)

sourceFOCband = ColumnDataSource({
        'base':calcPoundage,
        'bottom':bottomFOC,
        'normal':normalFOC,
        'high':highFOC,
        'extreme':extremeFOC,
        #'ultraextreme':ultraExtremeFOC,
        'top':topFOC
        })

bottomFOCband = Band(base='base', lower='bottom', upper='normal', source=sourceFOCband, fill_alpha=0.5, level='underlay', fill_color = "red")
normalFOCband = Band(base='base', lower='normal', upper='high', source=sourceFOCband, fill_alpha=0.5, level='underlay', fill_color = "#90CAF9")
highFOCband = Band(base='base', lower='high', upper='extreme', source=sourceFOCband, fill_alpha=0.5, level='underlay', fill_color = "#42A5F5")
extremeFOCband = Band(base='base', lower='extreme', upper='top', source=sourceFOCband, fill_alpha=0.5, level='underlay', fill_color = "#1E88E5")
#ultraExtremeFOCband = Band(base='base', lower='ultraextreme', upper='top', source=sourceFOCband, fill_alpha=0.5, level='underlay', fill_color = "#1565E0")

plotFOC.add_layout(bottomFOCband)
plotFOC.add_layout(normalFOCband)
plotFOC.add_layout(highFOCband)
plotFOC.add_layout(extremeFOCband)
#plotFOC.add_layout(ultraExtremeFOCband)


chosenSpine_slider = Slider(start=150, end=400, value=200, step=10.0, title="Arrow Shaft Spine")
chosenArrowGPI_slider = Slider(start=5, end=16, value=10.7, step=.1, title="Arrow Shaft GPI (gr/inch)")
chosenArrowLength_slider = Slider(start=24, end=32, value=28.25, step=.05, title="Arrow Shaft Length (inches)")
chosenArrowDiam_slider = Slider(start=0.166, end=0.300, value=0.166, step=.001, title="Arrow Diameter (inches)")

chosenPoundage_slider = Slider(start=30, end=90, value=71, step=1, title="Poundage")
chosenIBO_slider = Slider(start=300, end=360, value=335, step=1, title="IBO")
chosenDrawLength_slider = Slider(start=24, end=32, value=29, step=0.05, title="Draw Length")                                                                                                                     
                                                                                                                                                                                                         
chosenNockThroatAdder_slider = Slider(start=0, end=1, value=0.5, step=0.1, title="Nock Throat to Shaft Distance (inches)")
chosenNockWeight_slider = Slider(start=1, end=30, value=6, step=1, title="Total Nock Components Weight (grains)")

chosenArrowWrapWeight_slider = Slider(start=1, end=20, value=0, step=.1, title="Wrap Weight (grains)")
chosenArrowWrapLength_slider = Slider(start=1, end=10, value=4, step=1, title="Wrap Length (inches)")

chosenFletchNumber_slider = Slider(start=3, end=6, value=4, step=1, title="Number of Fletches")
chosenFletchDistanceFromShaftEnd_slider = Slider(start=0, end=2, value=0.75, step=0.05, title="Shaft End to Fletch Distance (inches)")
chosenFletchWeight_slider = Slider(start=1, end=10, value=5, step=.1, title="Weight per Fletch (grains)")
chosenFletchLength_slider = Slider(start=1, end=5, value=2.25, step=.05, title="Fletch Length (inches)")
chosenFletchHeight_slider = Slider(start=0.1, end=1, value=0.465, step=.01, title="Fletch Height (inches)")
chosenFletchOffset_slider = Slider(start=0, end=10, value=3, step=1, title="Fletch Offset Angle (degrees)")
chosenCoefDrag_slider = Slider(start=0.1, end=3, value=2, step=0.1, title="Coefficient of Drag")

#2nd Arrow-Bow Configuration

#Poundage [lbs]
calcPoundage2 = np.linspace(30, 90, 30)

#optimalPointWeight [gr]
calcOpPointWeight2 = 150+25/5 * (-0.252 * chosenIBO2 + 81.8 -calcPoundage2 + (aggregateRegValuesSlopeSlope*chosenArrowLength2 
                    + aggregateRegValuesSlopeIntercept) * chosenSpine2 
                    + aggregateRegValuesIntSlope*chosenArrowLength2 
                    + aggregateRegValuesIntIntercept)  
sourceOpPointWeight2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcOpPointWeight2))

#totalArrowMass [gr]
calcTotalArrowMass2 = chosenNockWeight2 + chosenArrowWrapWeight2 + chosenFletchNumber2 * chosenFletchWeight2 + chosenArrowGPI2 * chosenArrowLength2 + calcOpPointWeight2
sourceTotalArrowMass2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcTotalArrowMass2))

#FOC [%]

calcFOC2 = (100 *((chosenNockWeight2 * chosenNockThroatAdder2 + chosenArrowWrapLength2 * (chosenNockThroatAdder2 + chosenArrowWrapLength2/2) 
            + (chosenFletchNumber2 * chosenFletchWeight2) * (chosenFletchDistanceFromShaftEnd2 + chosenFletchLength2/3) 
            + (chosenArrowGPI2 * chosenArrowLength2) * (chosenNockThroatAdder2 + chosenArrowLength2/2) 
            + calcOpPointWeight2 * (chosenNockThroatAdder2 + chosenArrowLength2))
            /(calcTotalArrowMass2) 
            - (chosenArrowLength2 + chosenNockThroatAdder2)/2))/(chosenArrowLength2 + chosenNockThroatAdder2)     
sourceFOC2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcFOC2))

#Nominal Kinetic Energy [J]
calcKENominal2 = (0.5*((350/15.43)/1000)*((chosenIBO2-10*(30-chosenDrawLength2)-2*(70-calcPoundage2))*0.3048)**2)


#FPS [f/s]
calcMPS2 = ((calcKENominal2*2)/((calcTotalArrowMass2/15.43)/1000))**0.5
calcFPS2 = (((calcKENominal2*2)/((calcTotalArrowMass2/15.43)/1000))**0.5)/0.3048
sourceFPS2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcFPS2))

#Cross Sectional area of arrow
area_cross_section2 = np.pi*((chosenArrowDiam2/12)/2)**2 + chosenFletchNumber2 * 0.5 * chosenFletchLength2/12 * chosenFletchHeight2/12 * chosenFletchOffset2/90  # ft^2

#Velocity [fps]
calcFPS20yd2 = calculate_speed(calcFPS2, area_cross_section2, chosenCoefDrag2, calcTotalArrowMass2/7000, 60)
calcFPS40yd2 = calculate_speed(calcFPS2, area_cross_section2, chosenCoefDrag2, calcTotalArrowMass2/7000, 120)
calcFPS60yd2 = calculate_speed(calcFPS2, area_cross_section2, chosenCoefDrag2, calcTotalArrowMass2/7000, 180)
sourceFPS20yd2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcFPS20yd2))
sourceFPS40yd2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcFPS40yd2))
sourceFPS60yd2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcFPS60yd2))

#Time of Flight [s]
calcTOF20yd2 = calculate_time(calcFPS2, area_cross_section2, chosenCoefDrag2, calcTotalArrowMass2/7000, 60)
calcTOF40yd2 = calculate_time(calcFPS2, area_cross_section2, chosenCoefDrag2, calcTotalArrowMass2/7000, 120)
calcTOF60yd2 = calculate_time(calcFPS2, area_cross_section2, chosenCoefDrag2, calcTotalArrowMass2/7000, 180)
sourceTOF20yd2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcTOF20yd2))
sourceTOF40yd2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcTOF40yd2))
sourceTOF60yd2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcTOF60yd2))

#Kinetic Energy [J]
calcKE2 = 0.5*((calcMPS2**2)*((calcTotalArrowMass2/15.43)/1000))
sourceKE2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcKE2))
calcKE20yd2 = 0.5*(((calcFPS20yd2*0.3048)**2)*((calcTotalArrowMass2/15.43)/1000))
calcKE40yd2 = 0.5*(((calcFPS40yd2*0.3048)**2)*((calcTotalArrowMass2/15.43)/1000))
calcKE60yd2 = 0.5*(((calcFPS60yd2*0.3048)**2)*((calcTotalArrowMass2/15.43)/1000))
sourceKE20yd2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcKE20yd2))
sourceKE40yd2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcKE40yd2))
sourceKE60yd2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcKE60yd2))



#Momentum [kg*m/s]
calcMomentum2 = ((calcTotalArrowMass2/15.43)/1000)*calcMPS2
sourceMomentum2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcMomentum2))
calcMomentum20yd2 = ((calcTotalArrowMass2/15.43)/1000)*calcFPS20yd2*0.3048
calcMomentum40yd2 = ((calcTotalArrowMass2/15.43)/1000)*calcFPS40yd2*0.3048
calcMomentum60yd2 = ((calcTotalArrowMass2/15.43)/1000)*calcFPS60yd2*0.3048
sourceMomentum20yd2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcMomentum20yd2))
sourceMomentum40yd2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcMomentum40yd2))
sourceMomentum60yd2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcMomentum60yd2))



calcKEdotMomentum2 = calcKE2*calcMomentum2*calcFOC2
sourceKEdotMomentum2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcKEdotMomentum2))
calcKEdotMomentum20yd2 = calcKE20yd2*calcMomentum20yd2
calcKEdotMomentum40yd2 = calcKE40yd2*calcMomentum40yd2
calcKEdotMomentum60yd2 = calcKE60yd2*calcMomentum60yd2
sourceKEdotMomentum20yd2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcKEdotMomentum20yd2))
sourceKEdotMomentum40yd2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcKEdotMomentum40yd2))
sourceKEdotMomentum60yd2 = ColumnDataSource(data=dict(x=calcPoundage2, y=calcKEdotMomentum60yd2))


selectedPoundage2=chosenPoundage2
selectedOpPointWeight2=round(150+25/5 * (-0.252 * chosenIBO2 + 81.8 -selectedPoundage2 + (aggregateRegValuesSlopeSlope*chosenArrowLength2 + aggregateRegValuesSlopeIntercept) * chosenSpine2 + aggregateRegValuesIntSlope*chosenArrowLength2 + aggregateRegValuesIntIntercept),0)
selectedTotalArrowMass2=round(chosenNockWeight2 + chosenArrowWrapWeight2 + chosenFletchNumber2 * chosenFletchWeight2 + chosenArrowGPI2 * chosenArrowLength2 + selectedOpPointWeight2,0)
selectedFOC2=round((100 *((chosenNockWeight2 * chosenNockThroatAdder2 + chosenArrowWrapLength2 * (chosenNockThroatAdder2 + chosenArrowWrapLength2/2) + (chosenFletchNumber2 * chosenFletchWeight2) * (chosenFletchDistanceFromShaftEnd2 + chosenFletchLength2/3) + (chosenArrowGPI2 * chosenArrowLength2) * (chosenNockThroatAdder2 + chosenArrowLength2/2) + selectedOpPointWeight2 * (chosenNockThroatAdder2 + chosenArrowLength2))/(chosenNockWeight2 + chosenArrowWrapWeight2 + chosenFletchNumber2 * chosenFletchWeight2 + chosenArrowGPI2 * chosenArrowLength2 + selectedOpPointWeight2) - (chosenArrowLength2 + chosenNockThroatAdder2)/2))/(chosenArrowLength2 + chosenNockThroatAdder2),2)
selectedKENominal2=round((0.5*((350/15.43)/1000)*((chosenIBO2-10*(30-chosenDrawLength2)-2*(70-selectedPoundage2))*0.3048)**2),0)
selectedFPS2=round(((((0.5*((350/15.43)/1000)*((chosenIBO2-10*(30-chosenDrawLength2)-2*(70-selectedPoundage2))*0.3048)**2) * 2)/((selectedTotalArrowMass2/15.43)/1000))**0.5)/0.3048,0)
selectedKE2=round(0.5*(((selectedTotalArrowMass2/15.43)/1000)*(((0.5*((350/15.43)/1000)*((chosenIBO2-10*(30-chosenDrawLength2)-2*(70-selectedPoundage2))*0.3048)**2)*2)/((selectedTotalArrowMass2/15.43)/1000))),0)
selectedMomentum2=round(((selectedTotalArrowMass2/15.43)/1000)*(((0.5*((350/15.43)/1000)*(((chosenIBO2-10*(30-chosenDrawLength2)-2*(70-selectedPoundage2))*0.3048)**2)*2)/((selectedTotalArrowMass2/15.43)/1000))**0.5),2)
selectedKEdotMomentum2=round(selectedKE2*selectedMomentum2,0)
selectedFPS60yd2=round(calculate_speed(selectedFPS2, area_cross_section2, chosenCoefDrag2, selectedTotalArrowMass2/7000, 180),0)
selectedTOF20yd2 = round(calculate_time(selectedFPS2, area_cross_section2, chosenCoefDrag2, selectedTotalArrowMass2/7000, 60),3)
selectedTOF60yd2 = round(calculate_time(selectedFPS2, area_cross_section2, chosenCoefDrag2, selectedTotalArrowMass2/7000, 180),3)
selectedKE60yd2 = round(0.5*(((selectedFPS60yd2*0.3048)**2)*((selectedTotalArrowMass2/15.43)/1000)),0)
selectedMomentum60yd2 = round(((selectedTotalArrowMass2/15.43)/1000)*selectedFPS60yd2*0.3048,2)
selectedKEdotMomentum60yd2 = round(selectedKE60yd2*selectedMomentum60yd2,0)



sourceChosenPoundage2 = ColumnDataSource(data=dict(selectedPoundage2=[selectedPoundage2],
                                                  selectedOpPointWeight2=[selectedOpPointWeight2],
                                                  selectedTotalArrowMass2=[selectedTotalArrowMass2],
                                                  selectedFOC2=[selectedFOC2],
                                                  selectedFPS2=[selectedFPS2],
                                                  selectedKE2=[selectedKE2],
                                                  selectedMomentum2=[selectedMomentum2],
                                                  selectedKEdotMomentum2=[selectedKEdotMomentum2],
                                                  selectedFPS60yd2=[selectedFPS60yd2],
                                                  selectedTOF20yd2=[selectedTOF20yd2],
                                                  selectedTOF60yd2=[selectedTOF60yd2],
                                                  selectedKE60yd2=[selectedKE60yd2],
                                                  selectedMomentum60yd2=[selectedMomentum60yd2],
                                                  selectedKEdotMomentum60yd2=[selectedKEdotMomentum60yd2]
                                                 ))


labelOpPointWeight2 = LabelSet(x='selectedPoundage2', y='selectedOpPointWeight2', text=str('selectedOpPointWeight2'),
                  x_offset=10, y_offset=-25, source=sourceChosenPoundage2, text_color ="orange")
labelTotalArrowMass2 = LabelSet(x='selectedPoundage2', y='selectedTotalArrowMass2', text=str('selectedTotalArrowMass2'),
                  x_offset=10, y_offset=-25, source=sourceChosenPoundage2, text_color ="orange")
labelFOC2 = LabelSet(x='selectedPoundage2', y='selectedFOC2', text=str('selectedFOC2'),
                  x_offset=10, y_offset=-25, source=sourceChosenPoundage2, text_color ="orange")
labelFPS2 = LabelSet(x='selectedPoundage2', y='selectedFPS2', text=str('selectedFPS2'),
                  x_offset=10, y_offset=-25, source=sourceChosenPoundage2, text_color ="orange")
labelKE2 = LabelSet(x='selectedPoundage2', y='selectedKE2', text=str('selectedKE2'),
                  x_offset=10, y_offset=-25, source=sourceChosenPoundage2, text_color ="orange")
labelMomentum2 = LabelSet(x='selectedPoundage2', y='selectedMomentum2', text=str('selectedMomentum2'),
                  x_offset=10, y_offset=-25, source=sourceChosenPoundage2, text_color ="orange")
labelKEdotMomentum2 = LabelSet(x='selectedPoundage2', y='selectedKEdotMomentum2', text=str('selectedKEdotMomentum2'),
                  x_offset=10, y_offset=-25, source=sourceChosenPoundage2, text_color ="orange")
labelFPS60yd2 = LabelSet(x='selectedPoundage2', y='selectedFPS60yd2', text=str('selectedFPS60yd2'),
                  x_offset=10, y_offset=-25, source=sourceChosenPoundage2, text_color ="orange")
labelTOF20yd2 = LabelSet(x='selectedPoundage2', y='selectedTOF20yd2', text=str('selectedTOF20yd2'),
                  x_offset=10, y_offset=-25, source=sourceChosenPoundage2, text_color ="orange")
labelTOF60yd2 = LabelSet(x='selectedPoundage2', y='selectedTOF60yd2', text=str('selectedTOF60yd2'),
                  x_offset=10, y_offset=-25, source=sourceChosenPoundage2, text_color ="orange")
labelKE60yd2 = LabelSet(x='selectedPoundage2', y='selectedKE60yd2', text=str('selectedKE60yd2'),
                  x_offset=10, y_offset=-25, source=sourceChosenPoundage2, text_color ="orange")
labelMomentum60yd2 = LabelSet(x='selectedPoundage2', y='selectedMomentum60yd2', text=str('selectedMomentum60yd2'),
                  x_offset=10, y_offset=-25, source=sourceChosenPoundage2, text_color ="orange")
labelKEdotMomentum60yd2 = LabelSet(x='selectedPoundage2', y='selectedKEdotMomentum60yd', text=str('selectedKEdotMomentum60yd2'),
                  x_offset=10, y_offset=-25, source=sourceChosenPoundage2, text_color ="orange")

plotOpPointWeight.line('x', 'y', source=sourceOpPointWeight2, line_width=3, line_alpha=0.6, line_color = "orange")
plotTotalArrowMass.line('x', 'y', source=sourceTotalArrowMass2, line_width=3, line_alpha=0.6, line_color = "orange")
plotFOC.line('x', 'y', source=sourceFOC2, line_width=3, line_alpha=0.6, line_color = "orange")
plotFPS.line('x', 'y', source=sourceFPS2, line_width=3, line_alpha=0.6, line_color = "orange")
plotKE.line('x', 'y', source=sourceKE2, line_width=3, line_alpha=0.6, line_color = "orange")
plotMomentum.line('x', 'y', source=sourceMomentum2, line_width=3, line_alpha=0.6, line_color = "orange")
plotKEdotMomentum.line('x', 'y', source=sourceKEdotMomentum2, line_width=3, line_alpha=0.6, line_color = "orange")

plotFPS.line('x', 'y', source=sourceFPS20yd2, line_width=3, line_alpha=0.6, line_dash = "dashed", line_color = "orange")
plotFPS.line('x', 'y', source=sourceFPS40yd2, line_width=3, line_alpha=0.6, line_dash = "dashdot", line_color = "orange")
plotFPS.line('x', 'y', source=sourceFPS60yd2, line_width=3, line_alpha=0.6, line_dash = "dotted", line_color = "orange")
plotTOF.line('x', 'y', source=sourceTOF20yd2, line_width=3, line_alpha=0.6, line_dash = "dashed", line_color = "orange")
plotTOF.line('x', 'y', source=sourceTOF40yd2, line_width=3, line_alpha=0.6, line_dash = "dashdot", line_color = "orange")
plotTOF.line('x', 'y', source=sourceTOF60yd2, line_width=3, line_alpha=0.6, line_dash = "dotted", line_color = "orange")


plotKE.line('x', 'y', source=sourceKE20yd2, line_width=3, line_alpha=0.6, line_dash = "dashed", line_color = "orange")
plotMomentum.line('x', 'y', source=sourceMomentum20yd2, line_width=3, line_alpha=0.6, line_dash = "dashed", line_color = "orange")
plotKEdotMomentum.line('x', 'y', source=sourceKEdotMomentum20yd2, line_width=3, line_alpha=0.6, line_dash = "dashed", line_color = "orange")
plotKE.line('x', 'y', source=sourceKE40yd2, line_width=3, line_alpha=0.6, line_dash = "dashdot", line_color = "orange")
plotMomentum.line('x', 'y', source=sourceMomentum40yd2, line_width=3, line_alpha=0.6, line_dash = "dashdot", line_color = "orange")
plotKEdotMomentum.line('x', 'y', source=sourceKEdotMomentum40yd2, line_width=3, line_alpha=0.6, line_dash = "dashdot", line_color = "orange")
plotKE.line('x', 'y', source=sourceKE60yd2, line_width=3, line_alpha=0.6, line_dash = "dotted", line_color = "orange")
plotMomentum.line('x', 'y', source=sourceMomentum60yd2, line_width=3, line_alpha=0.6, line_dash = "dotted", line_color = "orange")
plotKEdotMomentum.line('x', 'y', source=sourceKEdotMomentum60yd2, line_width=3, line_alpha=0.6, line_dash = "dotted", line_color = "orange")


plotOpPointWeight.circle('selectedPoundage2','selectedOpPointWeight2', source=sourceChosenPoundage2, line_color="yellow", size=12, fill_color = "orange")
plotTotalArrowMass.circle('selectedPoundage2','selectedTotalArrowMass2', source=sourceChosenPoundage2, line_color="yellow", size=12, fill_color = "orange")
plotFOC.circle('selectedPoundage2','selectedFOC2', source=sourceChosenPoundage2, line_color="yellow", size=12, fill_color = "orange")
plotKE.circle('selectedPoundage2','selectedKE2', source=sourceChosenPoundage2, line_color="yellow", size=12, fill_color = "orange")
plotFPS.circle('selectedPoundage2','selectedFPS2', source=sourceChosenPoundage2, line_color="yellow", size=12, fill_color = "orange")
plotMomentum.circle('selectedPoundage2','selectedMomentum2', source=sourceChosenPoundage2, line_color="yellow", size=12, fill_color = "orange")
plotKEdotMomentum.circle('selectedPoundage2','selectedKEdotMomentum2', source=sourceChosenPoundage2, line_color="yellow", size=12, fill_color = "orange")
plotFPS.circle('selectedPoundage2','selectedFPS60yd2', source=sourceChosenPoundage2, line_color="yellow", size=12, fill_color = "orange")
plotTOF.circle('selectedPoundage2','selectedTOF20yd2', source=sourceChosenPoundage2, line_color="yellow", size=12, fill_color = "orange")
plotTOF.circle('selectedPoundage2','selectedTOF60yd2', source=sourceChosenPoundage2, line_color="yellow", size=12, fill_color = "orange")
plotKE.circle('selectedPoundage2','selectedKE60yd2', source=sourceChosenPoundage2, line_color="yellow", size=12, fill_color = "orange")
plotMomentum.circle('selectedPoundage2','selectedMomentum60yd2', source=sourceChosenPoundage2, line_color="yellow", size=12, fill_color = "orange")
plotKEdotMomentum.circle('selectedPoundage2','selectedKEdotMomentum60yd2', source=sourceChosenPoundage2, line_color="yellow", size=12, fill_color = "orange")



plotOpPointWeight.add_layout(labelOpPointWeight2)
plotTotalArrowMass.add_layout(labelTotalArrowMass2)
plotFOC.add_layout(labelFOC2)
plotFPS.add_layout(labelFPS2)
plotKE.add_layout(labelKE2)
plotMomentum.add_layout(labelMomentum2)
plotKEdotMomentum.add_layout(labelKEdotMomentum2)
plotFPS.add_layout(labelFPS60yd2)
plotTOF.add_layout(labelTOF20yd2)
plotTOF.add_layout(labelTOF60yd2)
plotKE.add_layout(labelKE60yd2)
plotMomentum.add_layout(labelMomentum60yd2)
plotKEdotMomentum.add_layout(labelKEdotMomentum60yd2)


chosenSpine_slider2 = Slider(start=150, end=400, value=300, step=10.0, title="Arrow Shaft Spine #2")
chosenArrowGPI_slider2 = Slider(start=5, end=16, value=7.1, step=.1, title="Arrow Shaft GPI (gr/inch) #2")
chosenArrowLength_slider2 = Slider(start=24, end=32, value=28.25, step=.05, title="Arrow Shaft Length (inches) #2")
chosenArrowDiam_slider2 = Slider(start=0.166, end=0.300, value=0.166, step=.001, title="Arrow Diameter (inches) #2")

chosenPoundage_slider2 = Slider(start=30, end=90, value=71, step=1, title="Poundage #2")
chosenIBO_slider2 = Slider(start=300, end=360, value=335, step=1, title="IBO #2")
chosenDrawLength_slider2 = Slider(start=24, end=32, value=29, step=0.05, title="Draw Length #2")                                                                                                                     
                                                                                                                                                                                                         
chosenNockThroatAdder_slider2 = Slider(start=0, end=1, value=0.5, step=0.1, title="Nock Throat to Shaft Distance (inches) #2")
chosenNockWeight_slider2 = Slider(start=1, end=30, value=6, step=1, title="Total Nock Components Weight (grains) #2")

chosenArrowWrapWeight_slider2 = Slider(start=1, end=20, value=0, step=.1, title="Wrap Weight (grains) #2")
chosenArrowWrapLength_slider2 = Slider(start=1, end=10, value=4, step=1, title="Wrap Length (inches) #2")

chosenFletchNumber_slider2 = Slider(start=3, end=6, value=4, step=1, title="Number of Fletches #2")
chosenFletchDistanceFromShaftEnd_slider2 = Slider(start=0, end=2, value=0.75, step=0.05, title="Shaft End to Fletch Distance (inches) #2")
chosenFletchWeight_slider2 = Slider(start=1, end=10, value=5, step=.1, title="Weight per Fletch (grains) #2")
chosenFletchLength_slider2 = Slider(start=1, end=5, value=2.25, step=.05, title="Fletch Length (inches) #2")
chosenFletchHeight_slider2 = Slider(start=0.1, end=1, value=0.465, step=.01, title="Fletch Height (inches) #2")
chosenFletchOffset_slider2 = Slider(start=0, end=10, value=3, step=1, title="Fletch Offset Angle (degrees) #2")
chosenCoefDrag_slider2 = Slider(start=0.1, end=3, value=2, step=0.1, title="Coefficient of Drag #2")    


#Set up real time updating of Plots
callback = CustomJS(args=dict(sourceChosenPoundage=sourceChosenPoundage,
                    sourceOpPointWeight=sourceOpPointWeight,
                    sourceTotalArrowMass=sourceTotalArrowMass,
                    sourceFOC=sourceFOC,
                    sourceKE=sourceKE,
                    sourceFPS=sourceFPS,
                    sourceMomentum=sourceMomentum,
                    sourceKEdotMomentum=sourceKEdotMomentum,
                    sourceFPS20yd=sourceFPS20yd,
                    sourceFPS40yd=sourceFPS40yd,
                    sourceFPS60yd=sourceFPS60yd,
                    sourceTOF20yd=sourceTOF20yd,
                    sourceTOF40yd=sourceTOF40yd,
                    sourceTOF60yd=sourceTOF60yd,
                    sourceKE20yd=sourceKE60yd,
                    sourceKE40yd=sourceKE20yd,          
                    sourceKE60yd=sourceKE40yd,
                    sourceMomentum20yd=sourceMomentum20yd,
                    sourceMomentum40yd=sourceMomentum40yd,
                    sourceMomentum60yd=sourceMomentum60yd,
                    sourceKEdotMomentum20yd=sourceKEdotMomentum20yd,
                    sourceKEdotMomentum40yd=sourceKEdotMomentum40yd,
                    sourceKEdotMomentum60yd=sourceKEdotMomentum60yd,
                    chosenSpineX = chosenSpine_slider,
                    chosenArrowGPIX = chosenArrowGPI_slider,
                    chosenArrowLengthX = chosenArrowLength_slider,
                    chosenPoundageX = chosenPoundage_slider,
                    chosenIBOX = chosenIBO_slider,
                    chosenDrawLengthX = chosenDrawLength_slider,
                    chosenNockThroatAdderX = chosenNockThroatAdder_slider,
                    chosenNockWeightX = chosenNockWeight_slider,
                    chosenArrowWrapWeightX = chosenArrowWrapWeight_slider,
                    chosenArrowWrapLengthX = chosenArrowWrapLength_slider,
                    chosenFletchNumberX = chosenFletchNumber_slider,
                    chosenFletchDistanceFromShaftEndX = chosenFletchDistanceFromShaftEnd_slider,
                    chosenFletchWeightX = chosenFletchWeight_slider,
                    chosenFletchLengthX = chosenFletchLength_slider,
                    chosenFletchHeightX = chosenFletchHeight_slider,
                    chosenArrowDiamX = chosenArrowDiam_slider,
                    chosenFletchOffsetX = chosenFletchOffset_slider,
                    chosenCoefDragX = chosenCoefDrag_slider,
                    sourceChosenPoundage2=sourceChosenPoundage2,
                    sourceOpPointWeight2=sourceOpPointWeight2,
                    sourceTotalArrowMass2=sourceTotalArrowMass2,
                    sourceFOC2=sourceFOC2,
                    sourceKE2=sourceKE2,
                    sourceFPS2=sourceFPS2,
                    sourceMomentum2=sourceMomentum2,
                    sourceKEdotMomentum2=sourceKEdotMomentum2,
                    sourceFPS20yd2=sourceFPS20yd2,
                    sourceFPS40yd2=sourceFPS40yd2,
                    sourceFPS60yd2=sourceFPS60yd2,
                    sourceTOF20yd2=sourceTOF20yd2,
                    sourceTOF40yd2=sourceTOF40yd2,
                    sourceTOF60yd2=sourceTOF60yd2,
                    sourceKE20yd2=sourceKE60yd2,
                    sourceKE40yd2=sourceKE20yd2,          
                    sourceKE60yd2=sourceKE40yd2,
                    sourceMomentum20yd2=sourceMomentum20yd2,
                    sourceMomentum40yd2=sourceMomentum40yd2,
                    sourceMomentum60yd2=sourceMomentum60yd2,
                    sourceKEdotMomentum20yd2=sourceKEdotMomentum20yd2,
                    sourceKEdotMomentum40yd2=sourceKEdotMomentum40yd2,
                    sourceKEdotMomentum60yd2=sourceKEdotMomentum60yd2,
                    chosenSpineX2 = chosenSpine_slider2,
                    chosenArrowGPIX2 = chosenArrowGPI_slider2,
                    chosenArrowLengthX2 = chosenArrowLength_slider2,
                    chosenPoundageX2 = chosenPoundage_slider2,
                    chosenIBOX2 = chosenIBO_slider2,
                    chosenDrawLengthX2 = chosenDrawLength_slider2,
                    chosenNockThroatAdderX2 = chosenNockThroatAdder_slider2,
                    chosenNockWeightX2 = chosenNockWeight_slider2,
                    chosenArrowWrapWeightX2 = chosenArrowWrapWeight_slider2,
                    chosenArrowWrapLengthX2 = chosenArrowWrapLength_slider2,
                    chosenFletchNumberX2 = chosenFletchNumber_slider2,
                    chosenFletchDistanceFromShaftEndX2 = chosenFletchDistanceFromShaftEnd_slider2,
                    chosenFletchWeightX2 = chosenFletchWeight_slider2,
                    chosenFletchLengthX2 = chosenFletchLength_slider2,
                    chosenFletchHeightX2 = chosenFletchHeight_slider2,
                    chosenArrowDiamX2 = chosenArrowDiam_slider2,
                    chosenFletchOffsetX2 = chosenFletchOffset_slider2,
                    chosenCoefDragX2 = chosenCoefDrag_slider2),
                    code="""
    console.trace("first")
    const data1 = sourceOpPointWeight.data;
    const calcPoundage = data1['x'];
    const calcOpPointWeight = data1['y'];
    const data2 = sourceTotalArrowMass.data;
    const calcTotalArrowMass = data2['y'];
    const data3 = sourceFOC.data;
    const calcFOC = data3['y'];
    const data4 = sourceKE.data;
    const calcKE = data4['y'];
    const data5 = sourceFPS.data;
    const calcFPS = data5['y'];
    const data6 = sourceMomentum.data;
    const calcMomentum = data6['y'];
    const data7 = sourceKEdotMomentum.data;
    const calcKEdotMomentum = data7['y'];
    const data8 = sourceFPS20yd.data;
    const calcFPS20yd = data8['y'];
    const data9 = sourceFPS40yd.data;
    const calcFPS40yd = data9['y'];
    const data10 = sourceFPS60yd.data;
    const calcFPS60yd = data10['y'];
    const data11 = sourceTOF20yd.data;
    const calcTOF20yd = data11['y'];
    const data12 = sourceTOF40yd.data;
    const calcTOF40yd = data12['y'];
    const data13 = sourceTOF60yd.data;
    const calcTOF60yd = data13['y'];
    const data14 = sourceKE20yd.data;
    const calcKE20yd = data14['y'];
    const data15 = sourceKE40yd.data;
    const calcKE40yd = data15['y'];
    const data16 = sourceKE60yd.data;
    const calcKE60yd = data16['y'];
    const data17 = sourceMomentum20yd.data;
    const calcMomentum20yd = data17['y'];
    const data18 = sourceMomentum40yd.data;
    const calcMomentum40yd = data18['y'];
    const data19 = sourceMomentum60yd.data;
    const calcMomentum60yd = data19['y'];
    const data20 = sourceKEdotMomentum20yd.data;
    const calcKEdotMomentum20yd = data20['y'];
    const data21 = sourceKEdotMomentum40yd.data;
    const calcKEdotMomentum40yd = data21['y'];
    const data22 = sourceKEdotMomentum60yd.data;
    const calcKEdotMomentum60yd = data22['y'];
    const chosenSpine = chosenSpineX.value;
    const chosenArrowGPI = chosenArrowGPIX.value;
    const chosenArrowLength = chosenArrowLengthX.value;
    const chosenPoundage = chosenPoundageX.value;
    const chosenIBO = chosenIBOX.value;
    const chosenDrawLength = chosenDrawLengthX.value;
    const chosenNockThroatAdder = chosenNockThroatAdderX.value;
    const chosenNockWeight = chosenNockWeightX.value;
    const chosenArrowWrapWeight = chosenArrowWrapWeightX.value;
    const chosenArrowWrapLength = chosenArrowWrapLengthX.value;
    const chosenFletchNumber = chosenFletchNumberX.value;
    const chosenFletchDistanceFromShaftEnd = chosenFletchDistanceFromShaftEndX.value;
    const chosenFletchWeight = chosenFletchWeightX.value;
    const chosenFletchLength = chosenFletchLengthX.value;
    const chosenFletchHeight = chosenFletchHeightX.value;
    const chosenArrowDiam = chosenArrowDiamX.value;
    const chosenFletchOffset = chosenFletchOffsetX.value;
    const chosenCoefDrag = chosenCoefDragX.value;
    const data101 = sourceOpPointWeight2.data;
    const calcPoundage2 = data101['x'];
    const calcOpPointWeight2 = data101['y'];
    const data102 = sourceTotalArrowMass2.data;
    const calcTotalArrowMass2 = data102['y'];
    const data103 = sourceFOC2.data;
    const calcFOC2 = data103['y'];
    const data104 = sourceKE2.data;
    const calcKE2 = data104['y'];
    const data105 = sourceFPS2.data;
    const calcFPS2 = data105['y'];
    const data106 = sourceMomentum2.data;
    const calcMomentum2 = data106['y'];
    const data107 = sourceKEdotMomentum2.data;
    const calcKEdotMomentum2 = data107['y'];
    const data108 = sourceFPS20yd2.data;
    const calcFPS20yd2 = data108['y'];
    const data109 = sourceFPS40yd2.data;
    const calcFPS40yd2 = data109['y'];
    const data110 = sourceFPS60yd2.data;
    const calcFPS60yd2 = data110['y'];
    const data111 = sourceTOF20yd2.data;
    const calcTOF20yd2 = data111['y'];
    const data112 = sourceTOF40yd2.data;
    const calcTOF40yd2 = data112['y'];
    const data113 = sourceTOF60yd2.data;
    const calcTOF60yd2 = data113['y'];
    const data114 = sourceKE20yd2.data;
    const calcKE20yd2 = data114['y'];
    const data115 = sourceKE40yd2.data;
    const calcKE40yd2 = data115['y'];
    const data116 = sourceKE60yd2.data;
    const calcKE60yd2 = data116['y'];
    const data117 = sourceMomentum20yd2.data;
    const calcMomentum20yd2 = data117['y'];
    const data118 = sourceMomentum40yd2.data;
    const calcMomentum40yd2 = data118['y'];
    const data119 = sourceMomentum60yd2.data;
    const calcMomentum60yd2 = data119['y'];
    const data120 = sourceKEdotMomentum20yd2.data;
    const calcKEdotMomentum20yd2 = data120['y'];
    const data121 = sourceKEdotMomentum40yd2.data;
    const calcKEdotMomentum40yd2 = data121['y'];
    const data122 = sourceKEdotMomentum60yd2.data;
    const calcKEdotMomentum60yd2 = data122['y'];
    const chosenSpine2 = chosenSpineX2.value;
    const chosenArrowGPI2 = chosenArrowGPIX2.value;
    const chosenArrowLength2 = chosenArrowLengthX2.value;
    const chosenPoundage2 = chosenPoundageX2.value;
    const chosenIBO2 = chosenIBOX2.value;
    const chosenDrawLength2 = chosenDrawLengthX2.value;
    const chosenNockThroatAdder2 = chosenNockThroatAdderX2.value;
    const chosenNockWeight2 = chosenNockWeightX2.value;
    const chosenArrowWrapWeight2 = chosenArrowWrapWeightX2.value;
    const chosenArrowWrapLength2 = chosenArrowWrapLengthX2.value;
    const chosenFletchNumber2 = chosenFletchNumberX2.value;
    const chosenFletchDistanceFromShaftEnd2 = chosenFletchDistanceFromShaftEndX2.value;
    const chosenFletchWeight2 = chosenFletchWeightX2.value;
    const chosenFletchLength2 = chosenFletchLengthX2.value;
    const chosenFletchHeight2 = chosenFletchHeightX2.value;
    const chosenArrowDiam2 = chosenArrowDiamX2.value;
    const chosenFletchOffset2 = chosenFletchOffsetX2.value;
    const chosenCoefDrag2 = chosenCoefDragX2.value;
    function calculate_speed(initial_velocity, area_cross_section, coefficient_drag, arrow_mass, distance) {
        const air_density = 0.0752; // lb/ft^3
        if (typeof initial_velocity === 'number') {
            // Calculate the velocity at the given distance
            const runtime = 3;
            let time_of_flight = 0;
            let distance_traveled = 0;
            const acceleration = (v) => -0.5 * air_density * area_cross_section * coefficient_drag * Math.pow(v, 2) / arrow_mass;
            let velocity = initial_velocity;
            const dt = 0.001; // time step size
            for (let t = 0; t < runtime / dt; t++) {
                velocity += acceleration(velocity) * dt;
                time_of_flight += dt;
                distance_traveled += velocity * dt;
                if (distance_traveled >= distance) {
                    break;
                }
            }
            return velocity;
        }
        const v_list = [];
        for (let i = 0; i < initial_velocity.length; i++) {
            const iv = initial_velocity[i];
            // Calculate the velocity at the given distance
            const runtime = 3;
            let time_of_flight = 0;
            let distance_traveled = 0;
            const acceleration = (v) => -0.5 * air_density * area_cross_section * coefficient_drag * Math.pow(v, 2) / arrow_mass[i];
            let velocity = iv;
            const dt = 0.001; // time step size
            for (let t = 0; t < runtime / dt; t++) {
                velocity += acceleration(velocity) * dt;
                time_of_flight += dt;
                distance_traveled += velocity * dt;
                if (distance_traveled >= distance) {
                    break;
                }
            }
            v_list.push(velocity);
        }
        // Return the velocity in feet per second
        return v_list;
    }
    function calculate_time(initial_velocity, area_cross_section, coefficient_drag, arrow_mass, distance) {
        const air_density = 0.0752; // lb/ft^3
        if (typeof initial_velocity === 'number') {
            const runtime = 3;
            let time_of_flight = 0;
            let distance_traveled = 0;
            const acceleration = (v) => -0.5 * air_density * area_cross_section * coefficient_drag * Math.pow(v, 2) / arrow_mass;
            let velocity = initial_velocity;
            const dt = 0.001; // time step size
            for (let t = 0; t < runtime / dt; t++) {
                velocity += acceleration(velocity) * dt;
                time_of_flight += dt;
                distance_traveled += velocity * dt;
                if (distance_traveled >= distance) {
                    break;
                }
            }
            return time_of_flight;
        }
        const t_list = [];
        for (let i = 0; i < initial_velocity.length; i++) {
            const iv = initial_velocity[i];
            const runtime = 3;
            let time_of_flight = 0;
            let distance_traveled = 0;
            const acceleration = (v) => -0.5 * air_density * area_cross_section * coefficient_drag * Math.pow(v, 2) / arrow_mass[i];
            let velocity = iv;
            const dt = 0.001; // time step size
            for (let t = 0; t < runtime / dt; t++) {
                velocity += acceleration(velocity) * dt;
                time_of_flight += dt;
                distance_traveled += velocity * dt;
                if (distance_traveled >= distance) {
                    break;
                }
            }
            t_list.push(time_of_flight);
        }
        // Return the time of flight in seconds
        return t_list;
    }
    let area_cross_section = Math.PI*((chosenArrowDiam/12)/2)**2 + chosenFletchNumber * 0.5 * chosenFletchLength/12 * chosenFletchHeight/12 * chosenFletchOffset/90;
    for (var i = 0; i < calcPoundage.length; i++) {
        calcOpPointWeight[i] = (150+25/5*(-0.252*chosenIBO+81.8-calcPoundage[i]+(-0.001*chosenArrowLength-0.174)*chosenSpine-3.885*chosenArrowLength+237.637));
        calcTotalArrowMass[i] = (chosenNockWeight + chosenArrowWrapWeight + chosenFletchNumber * chosenFletchWeight + chosenArrowGPI * chosenArrowLength + (150+25/5*(-0.252*chosenIBO+81.8-calcPoundage[i]+(-0.001*chosenArrowLength-0.174)*chosenSpine-3.885*chosenArrowLength+237.637)));
        calcFOC[i] =(100*((chosenNockWeight*chosenNockThroatAdder+chosenArrowWrapLength * (chosenNockThroatAdder + chosenArrowWrapLength/2) + (chosenFletchNumber * chosenFletchWeight) * (chosenFletchDistanceFromShaftEnd + chosenFletchLength/3) + (chosenArrowGPI * chosenArrowLength) * (chosenNockThroatAdder + chosenArrowLength/2) + calcOpPointWeight[i] * (chosenNockThroatAdder + chosenArrowLength))/(calcTotalArrowMass[i]) - (chosenArrowLength + chosenNockThroatAdder)/2))/(chosenArrowLength + chosenNockThroatAdder);
        calcFPS[i] = ((((0.5*((350/15.43)/1000)*((chosenIBO-10*(30-chosenDrawLength)-2*(70-calcPoundage[i]))*0.3048)**2)*2)/((calcTotalArrowMass[i]/15.43)/1000))**0.5)/0.3048;
        calcKE[i] = 0.5*(((calcTotalArrowMass[i]/15.43)/1000)*(((0.5*((350/15.43)/1000)*((chosenIBO-10*(30-chosenDrawLength)-2*(70-calcPoundage[i]))*0.3048)**2) * 2)/ ((calcTotalArrowMass[i]/15.43)/1000)));
        calcMomentum[i] = ((calcTotalArrowMass[i]/15.43)/1000)*((((0.5*((350/15.43)/1000)*((chosenIBO-10*(30-chosenDrawLength)-2*(70-calcPoundage[i]))*0.3048)**2) * 2)/ ((calcTotalArrowMass[i]/15.43)/1000))**0.5);
        calcKEdotMomentum[i] = calcKE[i]*calcMomentum[i];
        calcFPS20yd[i] = calculate_speed(calcFPS[i], area_cross_section, chosenCoefDrag, calcTotalArrowMass[i]/7000, 60);
        calcFPS40yd[i] = calculate_speed(calcFPS[i], area_cross_section, chosenCoefDrag, calcTotalArrowMass[i]/7000, 120);
        calcFPS60yd[i] = calculate_speed(calcFPS[i], area_cross_section, chosenCoefDrag, calcTotalArrowMass[i]/7000, 180);
        calcTOF20yd[i] = calculate_time(calcFPS[i], area_cross_section, chosenCoefDrag, calcTotalArrowMass[i]/7000, 60);
        calcTOF40yd[i] = calculate_time(calcFPS[i], area_cross_section, chosenCoefDrag, calcTotalArrowMass[i]/7000, 120);
        calcTOF60yd[i] = calculate_time(calcFPS[i], area_cross_section, chosenCoefDrag, calcTotalArrowMass[i]/7000, 180);
        calcKE20yd[i] = 0.5*(((calcFPS20yd[i]*0.3048)**2)*((calcTotalArrowMass[i]/15.43)/1000));
        calcKE40yd[i] = 0.5*(((calcFPS40yd[i]*0.3048)**2)*((calcTotalArrowMass[i]/15.43)/1000));
        calcKE60yd[i] = 0.5*(((calcFPS60yd[i]*0.3048)**2)*((calcTotalArrowMass[i]/15.43)/1000));
        calcMomentum20yd[i] = ((calcTotalArrowMass[i]/15.43)/1000)*calcFPS20yd[i]*0.3048;
        calcMomentum40yd[i] = ((calcTotalArrowMass[i]/15.43)/1000)*calcFPS40yd[i]*0.3048;
        calcMomentum60yd[i] = ((calcTotalArrowMass[i]/15.43)/1000)*calcFPS60yd[i]*0.3048;
        calcKEdotMomentum20yd[i] = calcKE20yd[i]*calcMomentum20yd[i];
        calcKEdotMomentum40yd[i] = calcKE40yd[i]*calcMomentum40yd[i];
        calcKEdotMomentum60yd[i] = calcKE60yd[i]*calcMomentum60yd[i];
    }
    let selectedPoundage = chosenPoundage;
    let selectedOpPointWeight = Math.round(150+25/5*(-0.252*chosenIBO+81.8-selectedPoundage+(-0.001*chosenArrowLength-0.174)*chosenSpine-3.885*chosenArrowLength+237.637));
    let selectedTotalArrowMass =Math.round(chosenNockWeight+chosenArrowWrapWeight+chosenFletchNumber*chosenFletchWeight+chosenArrowGPI*chosenArrowLength+selectedOpPointWeight);
    let selectedFOC = ((100*((chosenNockWeight*chosenNockThroatAdder+chosenArrowWrapLength*(chosenNockThroatAdder+chosenArrowWrapLength/2)+(chosenFletchNumber*chosenFletchWeight)*(chosenFletchDistanceFromShaftEnd+chosenFletchLength/3)+(chosenArrowGPI*chosenArrowLength)*(chosenNockThroatAdder+chosenArrowLength/2)+selectedOpPointWeight*(chosenNockThroatAdder+chosenArrowLength))/(chosenNockWeight+chosenArrowWrapWeight+chosenFletchNumber*chosenFletchWeight+chosenArrowGPI*chosenArrowLength+selectedOpPointWeight)-(chosenArrowLength+chosenNockThroatAdder)/2))/(chosenArrowLength+chosenNockThroatAdder)).toFixed(2);
    let selectedFPS = Math.round(((((0.5*((350/15.43)/1000)*((chosenIBO-10*(30-chosenDrawLength)-2*(70-selectedPoundage))*0.3048)**2)*2)/((selectedTotalArrowMass/15.43)/1000))**0.5)/0.3048);
    let selectedKE = Math.round(0.5*(((selectedTotalArrowMass/15.43)/1000)*(((0.5*((350/15.43)/1000)*((chosenIBO-10*(30-chosenDrawLength)-2*(70-selectedPoundage))*0.3048)**2)*2)/((selectedTotalArrowMass/15.43)/1000))));
    let selectedMomentum = (((selectedTotalArrowMass/15.43)/1000)*((((0.5*((350/15.43)/1000)*((chosenIBO-10*(30-chosenDrawLength)-2*(70-selectedPoundage))*0.3048)**2)*2)/((selectedTotalArrowMass/15.43)/1000))**0.5)).toFixed(2);    
    let selectedKEdotMomentum = Math.round(selectedKE*selectedMomentum);
    let selectedFPS60yd =Math.round(calculate_speed(selectedFPS, area_cross_section, chosenCoefDrag, selectedTotalArrowMass/7000, 180));
    let selectedTOF20yd =(calculate_time(selectedFPS, area_cross_section, chosenCoefDrag, selectedTotalArrowMass/7000, 60)).toFixed(3);
    let selectedTOF60yd =(calculate_time(selectedFPS, area_cross_section, chosenCoefDrag, selectedTotalArrowMass/7000, 180)).toFixed(3);
    let selectedKE60yd = Math.round(0.5*(((selectedFPS60yd*0.3048)**2)*((selectedTotalArrowMass/15.43)/1000)));
    let selectedMomentum60yd = (((selectedTotalArrowMass/15.43)/1000)*selectedFPS60yd*0.3048).toFixed(2);    
    let selectedKEdotMomentum60yd = Math.round(selectedKE60yd*selectedMomentum60yd);    
    sourceChosenPoundage.data['selectedPoundage'] = [selectedPoundage]
    sourceChosenPoundage.data['selectedOpPointWeight'] = [selectedOpPointWeight]
    sourceChosenPoundage.data['selectedTotalArrowMass'] = [selectedTotalArrowMass]
    sourceChosenPoundage.data['selectedFOC'] = [selectedFOC]
    sourceChosenPoundage.data['selectedFPS'] = [selectedFPS]
    sourceChosenPoundage.data['selectedKE'] = [selectedKE]
    sourceChosenPoundage.data['selectedMomentum'] = [selectedMomentum]
    sourceChosenPoundage.data['selectedKEdotMomentum'] = [selectedKEdotMomentum]
    sourceChosenPoundage.data['selectedFPS60yd'] = [selectedFPS60yd]
    sourceChosenPoundage.data['selectedTOF20yd'] = [selectedTOF20yd]
    sourceChosenPoundage.data['selectedTOF60yd'] = [selectedTOF60yd]
    sourceChosenPoundage.data['selectedKE60yd'] = [selectedKE60yd]
    sourceChosenPoundage.data['selectedMomentum60yd'] = [selectedMomentum60yd]
    sourceChosenPoundage.data['selectedKEdotMomentum60yd'] = [selectedMomentum60yd]
    sourceChosenPoundage.change.emit();
    sourceOpPointWeight.change.emit();
    sourceTotalArrowMass.change.emit();
    sourceFOC.change.emit();
    sourceFPS.change.emit();
    sourceKE.change.emit();
    sourceMomentum.change.emit();
    sourceKEdotMomentum.change.emit();
    sourceFPS20yd.change.emit();
    sourceFPS40yd.change.emit();
    sourceFPS60yd.change.emit();
    sourceTOF20yd.change.emit();
    sourceTOF40yd.change.emit();
    sourceTOF60yd.change.emit();
    sourceKE20yd.change.emit();
    sourceKE40yd.change.emit();
    sourceKE60yd.change.emit();
    sourceMomentum20yd.change.emit();
    sourceMomentum40yd.change.emit()
    sourceMomentum60yd.change.emit()
    sourceKEdotMomentum20yd.change.emit();
    sourceKEdotMomentum40yd.change.emit();
    sourceKEdotMomentum60yd.change.emit();
    let area_cross_section2 = Math.PI*((chosenArrowDiam2/12)/2)**2 + chosenFletchNumber2 * 0.5 * chosenFletchLength2/12 * chosenFletchHeight2/12 * chosenFletchOffset2/90;
    for (var i = 0; i < calcPoundage2.length; i++) {
        calcOpPointWeight2[i] = (150+25/5*(-0.252*chosenIBO2+81.8-calcPoundage2[i]+(-0.001*chosenArrowLength2-0.174)*chosenSpine2-3.885*chosenArrowLength2+237.637));
        calcTotalArrowMass2[i] = (chosenNockWeight2 + chosenArrowWrapWeight2 + chosenFletchNumber2 * chosenFletchWeight2 + chosenArrowGPI2 * chosenArrowLength2 + (150+25/5*(-0.252*chosenIBO2+81.8-calcPoundage2[i]+(-0.001*chosenArrowLength2-0.174)*chosenSpine2-3.885*chosenArrowLength2+237.637)));
        calcFOC2[i] =(100*((chosenNockWeight2*chosenNockThroatAdder2+chosenArrowWrapLength2 * (chosenNockThroatAdder2 + chosenArrowWrapLength2/2) + (chosenFletchNumber2 * chosenFletchWeight2) * (chosenFletchDistanceFromShaftEnd2 + chosenFletchLength2/3) + (chosenArrowGPI2 * chosenArrowLength2) * (chosenNockThroatAdder2 + chosenArrowLength2/2) + calcOpPointWeight2[i] * (chosenNockThroatAdder2 + chosenArrowLength2))/(calcTotalArrowMass2[i]) - (chosenArrowLength2 + chosenNockThroatAdder2)/2))/(chosenArrowLength2 + chosenNockThroatAdder2);
        calcFPS2[i] = ((((0.5*((350/15.43)/1000)*((chosenIBO2-10*(30-chosenDrawLength2)-2*(70-calcPoundage2[i]))*0.3048)**2)*2)/((calcTotalArrowMass2[i]/15.43)/1000))**0.5)/0.3048;
        calcKE2[i] = 0.5*(((calcTotalArrowMass2[i]/15.43)/1000)*(((0.5*((350/15.43)/1000)*((chosenIBO2-10*(30-chosenDrawLength2)-2*(70-calcPoundage2[i]))*0.3048)**2) * 2)/ ((calcTotalArrowMass2[i]/15.43)/1000)));
        calcMomentum2[i] = ((calcTotalArrowMass2[i]/15.43)/1000)*((((0.5*((350/15.43)/1000)*((chosenIBO2-10*(30-chosenDrawLength2)-2*(70-calcPoundage2[i]))*0.3048)**2) * 2)/ ((calcTotalArrowMass2[i]/15.43)/1000))**0.5);
        calcKEdotMomentum2[i] = calcKE2[i]*calcMomentum2[i];
        calcFPS20yd2[i] = calculate_speed(calcFPS2[i], area_cross_section2, chosenCoefDrag2, calcTotalArrowMass2[i]/7000, 60);
        calcFPS40yd2[i] = calculate_speed(calcFPS2[i], area_cross_section2, chosenCoefDrag2, calcTotalArrowMass2[i]/7000, 120);
        calcFPS60yd2[i] = calculate_speed(calcFPS2[i], area_cross_section2, chosenCoefDrag2, calcTotalArrowMass2[i]/7000, 180);
        calcTOF20yd2[i] = calculate_time(calcFPS2[i], area_cross_section2, chosenCoefDrag2, calcTotalArrowMass2[i]/7000, 60);
        calcTOF40yd2[i] = calculate_time(calcFPS2[i], area_cross_section2, chosenCoefDrag2, calcTotalArrowMass2[i]/7000, 120);
        calcTOF60yd2[i] = calculate_time(calcFPS2[i], area_cross_section2, chosenCoefDrag2, calcTotalArrowMass2[i]/7000, 180);
        calcKE20yd2[i] = 0.5*(((calcFPS20yd2[i]*0.3048)**2)*((calcTotalArrowMass2[i]/15.43)/1000));
        calcKE40yd2[i] = 0.5*(((calcFPS40yd2[i]*0.3048)**2)*((calcTotalArrowMass2[i]/15.43)/1000));
        calcKE60yd2[i] = 0.5*(((calcFPS60yd2[i]*0.3048)**2)*((calcTotalArrowMass2[i]/15.43)/1000));
        calcMomentum20yd2[i] = ((calcTotalArrowMass2[i]/15.43)/1000)*calcFPS20yd2[i]*0.3048;
        calcMomentum40yd2[i] = ((calcTotalArrowMass2[i]/15.43)/1000)*calcFPS40yd2[i]*0.3048;
        calcMomentum60yd2[i] = ((calcTotalArrowMass2[i]/15.43)/1000)*calcFPS60yd2[i]*0.3048;
        calcKEdotMomentum20yd2[i] = calcKE20yd2[i]*calcMomentum20yd2[i];
        calcKEdotMomentum40yd2[i] = calcKE40yd2[i]*calcMomentum40yd2[i];
        calcKEdotMomentum60yd2[i] = calcKE60yd2[i]*calcMomentum60yd2[i];
    }
    let selectedPoundage2 = chosenPoundage2;
    let selectedOpPointWeight2 = Math.round(150+25/5*(-0.252*chosenIBO2+81.8-selectedPoundage2+(-0.001*chosenArrowLength2-0.174)*chosenSpine2-3.885*chosenArrowLength2+237.637));
    let selectedTotalArrowMass2 =Math.round(chosenNockWeight2+chosenArrowWrapWeight2+chosenFletchNumber2*chosenFletchWeight2+chosenArrowGPI2*chosenArrowLength2+selectedOpPointWeight2);
    let selectedFOC2 = ((100*((chosenNockWeight2*chosenNockThroatAdder2+chosenArrowWrapLength2*(chosenNockThroatAdder2+chosenArrowWrapLength2/2)+(chosenFletchNumber2*chosenFletchWeight2)*(chosenFletchDistanceFromShaftEnd2+chosenFletchLength2/3)+(chosenArrowGPI2*chosenArrowLength2)*(chosenNockThroatAdder2+chosenArrowLength2/2)+selectedOpPointWeight2*(chosenNockThroatAdder2+chosenArrowLength2))/(chosenNockWeight2+chosenArrowWrapWeight2+chosenFletchNumber2*chosenFletchWeight2+chosenArrowGPI2*chosenArrowLength2+selectedOpPointWeight2)-(chosenArrowLength2+chosenNockThroatAdder2)/2))/(chosenArrowLength2+chosenNockThroatAdder2)).toFixed(2);
    let selectedFPS2 =Math.round(((((0.5*((350/15.43)/1000)*((chosenIBO2-10*(30-chosenDrawLength2)-2*(70-selectedPoundage2))*0.3048)**2)*2)/((selectedTotalArrowMass2/15.43)/1000))**0.5)/0.3048);
    let selectedKE2 = Math.round(0.5*(((selectedTotalArrowMass2/15.43)/1000)*(((0.5*((350/15.43)/1000)*((chosenIBO2-10*(30-chosenDrawLength2)-2*(70-selectedPoundage2))*0.3048)**2)*2)/((selectedTotalArrowMass2/15.43)/1000))));
    let selectedMomentum2 =(((selectedTotalArrowMass2/15.43)/1000)*((((0.5*((350/15.43)/1000)*((chosenIBO2-10*(30-chosenDrawLength2)-2*(70-selectedPoundage2))*0.3048)**2)*2)/((selectedTotalArrowMass2/15.43)/1000))**0.5)).toFixed(2);    
    let selectedKEdotMomentum2 = Math.round(selectedKE2*selectedMomentum2);
    let selectedFPS60yd2 =Math.round(calculate_speed(selectedFPS2, area_cross_section2, chosenCoefDrag2, selectedTotalArrowMass2/7000, 180));
    let selectedTOF20yd2 =(calculate_time(selectedFPS2, area_cross_section2, chosenCoefDrag2, selectedTotalArrowMass2/7000, 60)).toFixed(3);
    let selectedTOF60yd2 =(calculate_time(selectedFPS2, area_cross_section2, chosenCoefDrag2, selectedTotalArrowMass2/7000, 180)).toFixed(3);
    let selectedKE60yd2 = Math.round(0.5*(((selectedFPS60yd2*0.3048)**2)*((selectedTotalArrowMass2/15.43)/1000)));
    let selectedMomentum60yd2 = (((selectedTotalArrowMass2/15.43)/1000)*selectedFPS60yd2*0.3048).toFixed(2);    
    let selectedKEdotMomentum60yd2 = Math.round(selectedKE60yd2*selectedMomentum60yd2);    
    sourceChosenPoundage2.data['selectedPoundage2'] = [selectedPoundage2]
    sourceChosenPoundage2.data['selectedOpPointWeight2'] = [selectedOpPointWeight2]
    sourceChosenPoundage2.data['selectedTotalArrowMass2'] = [selectedTotalArrowMass2]
    sourceChosenPoundage2.data['selectedFOC2'] = [selectedFOC2]
    sourceChosenPoundage2.data['selectedFPS2'] = [selectedFPS2]
    sourceChosenPoundage2.data['selectedKE2'] = [selectedKE2]
    sourceChosenPoundage2.data['selectedMomentum2'] = [selectedMomentum2]
    sourceChosenPoundage2.data['selectedKEdotMomentum2'] = [selectedKEdotMomentum2]
    sourceChosenPoundage2.data['selectedFPS60yd2'] = [selectedFPS60yd2]
    sourceChosenPoundage2.data['selectedTOF20yd2'] = [selectedTOF20yd2]
    sourceChosenPoundage2.data['selectedTOF60yd2'] = [selectedTOF60yd2]
    sourceChosenPoundage2.data['selectedKE60yd2'] = [selectedKE60yd2]
    sourceChosenPoundage2.data['selectedMomentum60yd2'] = [selectedMomentum60yd2]
    sourceChosenPoundage2.data['selectedKEdotMomentum60yd2'] = [selectedMomentum60yd2]
    sourceChosenPoundage2.change.emit();
    sourceOpPointWeight2.change.emit();
    sourceTotalArrowMass2.change.emit();
    sourceFOC2.change.emit();
    sourceFPS2.change.emit();
    sourceKE2.change.emit();
    sourceMomentum2.change.emit();
    sourceKEdotMomentum2.change.emit();
    sourceFPS20yd2.change.emit();
    sourceFPS40yd2.change.emit();
    sourceFPS60yd2.change.emit();
    sourceTOF20yd2.change.emit();
    sourceTOF40yd2.change.emit();
    sourceTOF60yd2.change.emit();
    sourceKE20yd2.change.emit();
    sourceKE40yd2.change.emit();
    sourceKE60yd2.change.emit();
    sourceMomentum20yd2.change.emit();
    sourceMomentum40yd2.change.emit()
    sourceMomentum60yd2.change.emit()
    sourceKEdotMomentum20yd2.change.emit();
    sourceKEdotMomentum40yd2.change.emit();
    sourceKEdotMomentum60yd2.change.emit();
""") 


chosenSpine_slider.js_on_change('value', callback)
chosenArrowGPI_slider.js_on_change('value', callback)
chosenArrowLength_slider.js_on_change('value', callback)
chosenArrowDiam_slider.js_on_change('value', callback)

chosenPoundage_slider.js_on_change('value', callback)
chosenIBO_slider.js_on_change('value', callback)
chosenDrawLength_slider.js_on_change('value', callback)                                                                                                                     

chosenNockThroatAdder_slider.js_on_change('value', callback)
chosenNockWeight_slider.js_on_change('value', callback)

chosenArrowWrapWeight_slider.js_on_change('value', callback)
chosenArrowWrapLength_slider.js_on_change('value', callback)

chosenFletchNumber_slider.js_on_change('value', callback)
chosenFletchDistanceFromShaftEnd_slider.js_on_change('value', callback)
chosenFletchWeight_slider.js_on_change('value', callback)
chosenFletchLength_slider.js_on_change('value', callback)
chosenFletchHeight_slider.js_on_change('value', callback)
chosenFletchOffset_slider.js_on_change('value', callback)
chosenCoefDrag_slider.js_on_change('value', callback)


#2nd set of sliders
chosenSpine_slider2.js_on_change('value', callback)
chosenArrowGPI_slider2.js_on_change('value', callback)
chosenArrowLength_slider2.js_on_change('value', callback)
chosenArrowDiam_slider2.js_on_change('value', callback)

chosenPoundage_slider2.js_on_change('value', callback)
chosenIBO_slider2.js_on_change('value', callback)
chosenDrawLength_slider2.js_on_change('value', callback)                                                                                                                     

chosenNockThroatAdder_slider2.js_on_change('value', callback)
chosenNockWeight_slider2.js_on_change('value', callback)

chosenArrowWrapWeight_slider2.js_on_change('value', callback)
chosenArrowWrapLength_slider2.js_on_change('value', callback)

chosenFletchNumber_slider2.js_on_change('value', callback)
chosenFletchDistanceFromShaftEnd_slider2.js_on_change('value', callback)
chosenFletchWeight_slider2.js_on_change('value', callback)
chosenFletchLength_slider2.js_on_change('value', callback)
chosenFletchHeight_slider2.js_on_change('value', callback)
chosenFletchOffset_slider2.js_on_change('value', callback)
chosenCoefDrag_slider2.js_on_change('value', callback)


# show(spineradio_button_group)
layout3 = row(
    column(chosenPoundage_slider,
           chosenIBO_slider,
           chosenDrawLength_slider,
           chosenSpine_slider,
           chosenArrowGPI_slider,
           chosenArrowLength_slider,
           chosenArrowDiam_slider,
           chosenNockThroatAdder_slider,
           chosenNockWeight_slider,
           chosenArrowWrapWeight_slider,
           chosenArrowWrapLength_slider,
           chosenFletchNumber_slider,
           chosenFletchDistanceFromShaftEnd_slider,
           chosenFletchWeight_slider,
           chosenFletchLength_slider,
           chosenFletchHeight_slider,
           chosenFletchOffset_slider,
           chosenCoefDrag_slider,
           ),
    column(chosenPoundage_slider2,
           chosenIBO_slider2,
           chosenDrawLength_slider2,
           chosenSpine_slider2,
           chosenArrowGPI_slider2,
           chosenArrowLength_slider2,
           chosenArrowDiam_slider2,
           chosenNockThroatAdder_slider2,
           chosenNockWeight_slider2,
           chosenArrowWrapWeight_slider2,
           chosenArrowWrapLength_slider2,
           chosenFletchNumber_slider2,
           chosenFletchDistanceFromShaftEnd_slider2,
           chosenFletchWeight_slider2,
           chosenFletchLength_slider2,
           chosenFletchHeight_slider2,
           chosenFletchOffset_slider2,
           chosenCoefDrag_slider2,
           ),
    column(plotOpPointWeight,plotTotalArrowMass,plotFOC),
    column(plotFPS,plotTOF),
    column(plotMomentum,plotKE),
)
show(layout3)

#Chosen Setup: Sirius Orion with 2.25 Tac Driver. 4mm microlite Deep six G nock

Loading BokehJS ...